In [1]:
"""
============================================
SETUP INICIAL PARA GOOGLE COLAB
============================================
Ejecutar esta celda primero en Google Colab
"""

# ==================== INSTALACIÓN DE DEPENDENCIAS ====================
print("📦 Instalando dependencias...\n")

import sys
import warnings
warnings.filterwarnings('ignore')

# Instalar todas las librerías necesarias
print("📦 Instalando librerías...")
!pip install -q chromadb sentence-transformers rank-bm25 groq neo4j scikit-learn rarfile

print("✅ Dependencias instaladas\n")

# ==================== INSTALAR UNRAR ====================
print("📦 Instalando herramientas de sistema...")
!apt-get install -qq unrar > /dev/null 2>&1
print("✅ Herramientas instaladas\n")

# ==================== DESCOMPRIMIR FUENTES.RAR ====================
print("📦 Procesando fuentes de datos...")

import rarfile
from pathlib import Path
import os
import shutil

rar_path = Path('/content/fuentes.rar')

if rar_path.exists():
    print(f"✅ Archivo {rar_path.name} encontrado")

    data_dir = Path('/content/data')
    if data_dir.exists():
        print(f"🗑️ Limpiando carpeta data existente...")
        shutil.rmtree(data_dir)

    data_dir.mkdir(exist_ok=True)

    print(f"📂 Descomprimiendo en {data_dir}...")

    try:
        with rarfile.RarFile(rar_path, 'r') as rar_ref:
            rar_ref.extractall(data_dir)

        print("✅ Archivos descomprimidos")

        # Reorganizar estructura si es necesario
        subdirs = list(data_dir.glob('*/'))
        if len(subdirs) == 1:
            print("📁 Reorganizando estructura de carpetas...")
            subdir = subdirs[0]
            for item in subdir.iterdir():
                shutil.move(str(item), str(data_dir / item.name))
            subdir.rmdir()
            print("✅ Estructura reorganizada")

        # Consolidar archivos de subdirectorios
        print(f"\n📦 Consolidando archivos en /content/data/...")

        for pattern in ['*.csv', '*.json', '*.txt', '*.md']:
            for file in data_dir.rglob(pattern):
                if file.parent != data_dir:
                    dest = data_dir / file.name
                    if not dest.exists():
                        shutil.move(str(file), str(dest))

        # Limpiar directorios vacíos
        for subdir in list(data_dir.glob('*/')):
            if not list(subdir.iterdir()):
                subdir.rmdir()

        print("✅ Archivos consolidados")

        # Contar archivos
        csv_count = len(list(data_dir.glob('*.csv')))
        json_count = len(list(data_dir.glob('*.json')))
        txt_count = len(list(data_dir.glob('*.txt')))
        md_count = len(list(data_dir.glob('*.md')))

        print(f"\n📊 Resumen en /content/data/:")
        print(f"   📊 {csv_count} archivos CSV")
        print(f"   📄 {json_count} archivos JSON")
        print(f"   📝 {txt_count} archivos TXT")
        print(f"   📘 {md_count} archivos MD")

    except Exception as e:
        print(f"❌ ERROR: {e}")

else:
    print("⚠️ Archivo fuentes.rar NO encontrado en /content/")
    print("\n   INSTRUCCIONES:")
    print("   1. Haz clic en el ícono de carpeta 📁 en el panel izquierdo")
    print("   2. Arrastra o sube el archivo fuentes.rar")
    print("   3. Ejecuta esta celda nuevamente")

print()

# ==================== CONFIGURAR PATHS ====================
print("📍 Configurando rutas de trabajo...")

BASE_DIR = Path('/content')
DATA_DIR = Path('/content/data')
MODELS_DIR = Path('/content/models')
OUTPUT_DIR = Path('/content/outputs')

for dir_path in [MODELS_DIR, OUTPUT_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

print(f"✅ Rutas configuradas:")
print(f"   Base: {BASE_DIR}")
print(f"   Datos: {DATA_DIR}")
print(f"   Modelos: {MODELS_DIR}")
print(f"   Outputs: {OUTPUT_DIR}\n")

# ==================== VERIFICAR ARCHIVOS ====================
print("🔍 Verificando archivos de datos...")

csv_files = list(DATA_DIR.glob('*.csv'))
json_files = list(DATA_DIR.glob('*.json'))
txt_files = list(DATA_DIR.glob('*.txt'))
md_files = list(DATA_DIR.glob('*.md'))

print(f"✅ Archivos CSV: {len(csv_files)}")
for f in sorted(csv_files)[:5]:
    print(f"   📊 {f.name}")
if len(csv_files) > 5:
    print(f"   ... y {len(csv_files) - 5} más")

print(f"\n✅ Archivos JSON: {len(json_files)}")
for f in sorted(json_files):
    print(f"   📄 {f.name}")

if txt_files:
    print(f"\n✅ Archivos TXT: {len(txt_files)} (reseñas)")
    print(f"   Primeros 5: {[f.name for f in sorted(txt_files)[:5]]}")

if md_files:
    print(f"\n✅ Archivos MD: {len(md_files)} (manuales)")
    print(f"   Primeros 5: {[f.name for f in sorted(md_files)[:5]]}")

# Verificar archivos críticos
archivos_criticos = [
    'faqs.json',
    'productos.csv',
    'vendedores.csv',
    'ventas_historicas.csv',
    'tickets_soporte.csv',
    'inventario_sucursales.csv',
    'devoluciones.csv'
]

print(f"\n🔍 Verificando archivos críticos:")
archivos_faltantes = []
for archivo in archivos_criticos:
    if (DATA_DIR / archivo).exists():
        print(f"   ✅ {archivo}")
    else:
        print(f"   ❌ {archivo} - NO ENCONTRADO")
        archivos_faltantes.append(archivo)

if archivos_faltantes:
    print(f"\n⚠️ ADVERTENCIA: Faltan {len(archivos_faltantes)} archivos críticos")
else:
    print(f"\n✅ Todos los archivos críticos están disponibles")

print()

# ==================== RESUMEN FINAL ====================
print("="*60)
print("✅ SETUP COMPLETADO")
print("="*60)
print(f"📂 Directorio de datos: {DATA_DIR}")
print(f"📊 Archivos CSV: {len(csv_files)}")
print(f"📄 Archivos JSON: {len(json_files)}")
print(f"📝 Archivos TXT: {len(txt_files)}")
print(f"📘 Archivos MD: {len(md_files)}")
print("="*60)
print("\n✅ Listo para continuar con la siguiente celda")

📦 Instalando dependencias...

📦 Instalando librerías...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [2]:
"""
============================================
CELDA 2: CONFIGURACIÓN DE API KEYS Y CONFIG
============================================
"""

import os
from pathlib import Path

# ==================== CONFIGURAR API KEYS ====================
print("🔐 Configurando API Keys...")

# Cargar desde archivo .env si existe
env_path = '/content/.env' if os.path.exists('/content/.env') else '.env'

if os.path.exists(env_path):
    with open(env_path) as f:
        for line in f:
            if '=' in line and not line.startswith('#'):
                key, value = line.strip().split('=', 1)
                os.environ[key] = value
    print(f"✅ Credenciales cargadas desde {env_path}")
else:
    print("⚠️ Archivo .env no encontrado")
    print("   Crear archivo .env con las siguientes variables:")
    print("   GROQ_API_KEY=tu_clave")
    print("   NEO4J_URI=neo4j://tu_servidor:7687")
    print("   NEO4J_USER=neo4j")
    print("   NEO4J_PASSWORD=tu_password")

# Asignar variables desde entorno
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')
NEO4J_URI = os.environ.get('NEO4J_URI')
NEO4J_USER = os.environ.get('NEO4J_USER')
NEO4J_PASSWORD = os.environ.get('NEO4J_PASSWORD')

# Verificar credenciales
if all([GROQ_API_KEY, NEO4J_URI, NEO4J_PASSWORD]):
    print("✅ Todas las credenciales configuradas")
else:
    print("❌ Faltan credenciales - revisar archivo .env")

print("✅ Variables de entorno configuradas\n")

# ==================== CONFIGURACIÓN GLOBAL ====================

class Config:
    """Clase de configuración global del sistema RAG"""

    # ===== PATHS =====
    BASE_DIR = BASE_DIR
    DATA_DIR = DATA_DIR
    MODELS_DIR = MODELS_DIR
    OUTPUT_DIR = OUTPUT_DIR

    # ===== API KEYS =====
    GROQ_API_KEY = GROQ_API_KEY
    NEO4J_URI = NEO4J_URI
    NEO4J_USER = NEO4J_USER
    NEO4J_PASSWORD = NEO4J_PASSWORD

    # ===== ARCHIVOS DE DATOS =====
    DATA_FILES = {
        'faqs': 'faqs.json',
        'productos': 'productos.csv',
        'vendedores': 'vendedores.csv',
        'ventas': 'ventas_historicas.csv',
        'tickets': 'tickets_soporte.csv',
        'inventario': 'inventario_sucursales.csv',
        'devoluciones': 'devoluciones.csv'
    }

    # ===== PATRONES DE ARCHIVOS =====
    RESENAS_PATTERN = 'resena_*.txt'  # resena_R00001.txt, etc.
    MANUALES_PATTERN = 'manual_*.md'  # manual_P0001_*.md, etc.

    # ===== CONFIGURACIÓN LLM (GROQ) =====
    LLM_CONFIG = {
        'provider': 'groq',
        'model': 'llama-3.1-8b-instant',  # Modelo recomendado de GROQ. Se cambia el 70b-versatile por problemas de Rate Limit
        'temperature': 0.3,
        'max_tokens': 1024,
        'top_p': 0.9
    }

    # ===== CONFIGURACIÓN EMBEDDINGS =====
    EMBEDDING_CONFIG = {
        'model_name': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
        'device': 'cpu',  # Cambiar a 'cuda' si hay GPU disponible
        'normalize_embeddings': True
    }

    # ===== CONFIGURACIÓN CHROMADB =====
    CHROMA_CONFIG = {
        'collection_name': 'electrodomesticos_docs',
        'persist_directory': str(MODELS_DIR / 'chromadb'),
        'distance_metric': 'cosine'
    }

    # ===== CONFIGURACIÓN TEXT SPLITTER =====
    SPLITTER_CONFIG = {
        'chunk_size': 400,
        'chunk_overlap': 50,
        'separators': ["\n\n", "\n", ". ", " ", ""]
    }

    # ===== CONFIGURACIÓN BÚSQUEDA HÍBRIDA =====
    HYBRID_SEARCH_CONFIG = {
        'semantic_top_k': 20,
        'bm25_top_k': 20,
        'rerank_top_k': 5,
        'alpha': 0.7  # 70% semántico, 30% BM25
    }

    # ===== CONFIGURACIÓN MEMORIA =====
    MEMORY_CONFIG = {
        'max_turns': 5,
        'max_context_length': 2000
    }

    # ===== CLASES DE INTENCIÓN =====
    INTENT_CLASSES = {
        'vectorial': {
            'description': 'Preguntas sobre uso, funcionamiento, problemas, mantenimiento, opiniones',
            'keywords': ['cómo', 'usar', 'funciona', 'problema', 'opinión', 'reseña', 'manual']
        },
        'tabular': {
            'description': 'Consultas de precios, filtros, stock, especificaciones técnicas',
            'keywords': ['precio', 'menos de', 'mayor que', 'stock', 'disponible', 'cuánto']
        },
        'grafo': {
            'description': 'Productos relacionados, compatibilidad, accesorios, similares',
            'keywords': ['relacionado', 'compatible', 'similar', 'repuesto', 'accesorio']
        }
    }

    # ===== PROMPTS DEL SISTEMA =====
    SYSTEM_PROMPTS = {
        'rag_assistant': """Eres un asistente virtual experto en electrodomésticos.

Instrucciones:
1. Responde SIEMPRE en español
2. Sé claro, conciso y útil
3. Si no tienes información suficiente, sugiere reformular la pregunta
4. Cita información específica cuando sea relevante (ej: "El producto P0001...")
5. Mantén un tono profesional pero amigable
""",

        'filter_generator': """Eres un experto en generar filtros de búsqueda para bases de datos.

IMPORTANTE:
- Responde SOLO con un objeto JSON válido
- NO agregues explicaciones antes o después del JSON
- NO uses markdown (```json)
- El JSON debe ser parseable directamente con json.loads()
""",

        'cypher_generator': """Eres un experto en Neo4j y el lenguaje Cypher.

IMPORTANTE:
- Responde SOLO con la query Cypher
- NO agregues explicaciones
- La query debe ser ejecutable directamente
- Usa LIMIT para evitar resultados excesivos (máximo 50)
"""
    }

# Crear instancia de configuración global
config = Config()

print("="*60)
print("✅ CONFIGURACIÓN COMPLETADA")
print("="*60)
print(f"📂 Directorio de datos: {config.DATA_DIR}")
print(f"🤖 Proveedor LLM: {config.LLM_CONFIG['provider'].upper()}")
print(f"📝 Modelo: {config.LLM_CONFIG['model']}")
print(f"🧠 Modelo embeddings: {config.EMBEDDING_CONFIG['model_name']}")
print(f"💾 ChromaDB: {config.CHROMA_CONFIG['persist_directory']}")
print(f"🔐 API Keys configuradas: {'✅' if config.GROQ_API_KEY else '❌'}")
print("="*60 + "\n")

🔐 Configurando API Keys...
✅ Credenciales cargadas desde /content/.env
✅ Todas las credenciales configuradas
✅ Variables de entorno configuradas

✅ CONFIGURACIÓN COMPLETADA
📂 Directorio de datos: /content/data
🤖 Proveedor LLM: GROQ
📝 Modelo: llama-3.3-70b-versatile
🧠 Modelo embeddings: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
💾 ChromaDB: /content/models/chromadb
🔐 API Keys configuradas: ✅



In [3]:
"""
============================================
CELDA 3: CARGA DE DATOS
============================================
Carga y procesa todos los archivos de fuentes de información
"""

import json
import pandas as pd
from pathlib import Path
from typing import Dict, List, Tuple
import re

# ==================== CORRECCIÓN DE ENCODING ====================

def fix_encoding(text: str) -> str:
    """Corrige el encoding UTF-8 mal interpretado"""
    if not isinstance(text, str):
        return text

    # Usar códigos unicode en lugar de caracteres literales
    replacements = {
        '\xc3\xa1': 'á',  # á
        '\xc3\xa9': 'é',  # é
        '\xc3\xad': 'í',  # í
        '\xc3\xb3': 'ó',  # ó
        '\xc3\xba': 'ú',  # ú
        '\xc3\x81': 'Á',  # Á
        '\xc3\x89': 'É',  # É
        '\xc3\x8d': 'Í',  # Í
        '\xc3\x93': 'Ó',  # Ó
        '\xc3\x9a': 'Ú',  # Ú
        '\xc3\xb1': 'ñ',  # ñ
        '\xc3\x91': 'Ñ',  # Ñ
        '\xc2\xbf': '¿',  # ¿
        '\xc2\xa1': '¡',  # ¡
    }

    for bad, good in replacements.items():
        text = text.replace(bad, good)

    # Patrones adicionales comunes
    text = text.replace('\xc3\xb3n', 'ón')
    text = text.replace('\xc3\xadn', 'ín')
    text = text.replace('\xc3\xb1o', 'ño')

    return text


# ==================== CARGA DE ARCHIVOS ====================

def load_json(filepath: Path) -> List[Dict]:
    """Carga archivo JSON con corrección de encoding"""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            data = json.load(f)
    except UnicodeDecodeError:
        with open(filepath, 'r', encoding='latin-1') as f:
            data = json.load(f)

    if isinstance(data, list):
        for item in data:
            if isinstance(item, dict):
                for key, value in item.items():
                    if isinstance(value, str):
                        item[key] = fix_encoding(value)

    return data

def load_csv(filepath: Path) -> pd.DataFrame:
    """Carga archivo CSV con corrección de encoding y tipos de datos"""
    try:
        df = pd.read_csv(filepath, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(filepath, encoding='latin-1')

    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].apply(lambda x: fix_encoding(x) if isinstance(x, str) else x)

    # CORRECCIÓN DE TIPOS DE DATOS
    # Convertir columnas numéricas que pueden estar como texto
    numeric_columns = ['precio_usd', 'stock', 'potencia_w', 'peso_kg', 'garantia_meses',
                      'cantidad', 'total', 'descuento_pct', 'precio_unitario',
                      'stock_sucursal', 'precio_sucursal', 'monto_devuelto']

    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

def load_resenas(base_dir: Path) -> List[Dict]:
    """
    Carga todas las reseñas desde archivos .txt
    Busca en:
    - /content/data/resena_*.txt
    - /content/data/reseñas_usuarios/resena_*.txt
    """
    resenas = []

    # Buscar en múltiples ubicaciones
    search_paths = [
        base_dir / 'resena_*.txt',
        base_dir / 'reseñas_usuarios' / 'resena_*.txt',
        base_dir / 'resenas_usuarios' / 'resena_*.txt'
    ]

    txt_files = []
    for search_path in search_paths:
        txt_files.extend(list(base_dir.glob(str(search_path.relative_to(base_dir)))))

    # También búsqueda recursiva general
    txt_files.extend(list(base_dir.rglob('resena_*.txt')))

    # Eliminar duplicados
    txt_files = list(set(txt_files))

    print(f"   📝 Encontrados {len(txt_files)} archivos .txt")

    for txt_file in txt_files:
        match = re.search(r'resena_(\w+)\.txt', txt_file.name)
        if match:
            id_resena = match.group(1)

            try:
                with open(txt_file, 'r', encoding='utf-8') as f:
                    texto = f.read().strip()

                if texto:
                    resenas.append({
                        'id_resena': id_resena,
                        'texto': fix_encoding(texto),
                        'archivo': txt_file.name
                    })
            except Exception as e:
                print(f"      ⚠️ Error leyendo {txt_file.name}: {e}")

    return resenas

def parse_manual_sections(markdown_text: str) -> List[Dict]:
    """Parsea un manual en markdown y extrae secciones"""
    secciones = []

    partes = re.split(r'\n## ', markdown_text)

    for i, parte in enumerate(partes):
        if i == 0:
            continue

        lines = parte.split('\n', 1)
        titulo = lines[0].strip()
        contenido = lines[1].strip() if len(lines) > 1 else ""

        tipo = categorize_section(titulo)

        if contenido and tipo:
            secciones.append({
                'tipo': tipo,
                'titulo': titulo,
                'contenido': contenido
            })

    return secciones

def categorize_section(titulo: str) -> str:
    """Categoriza una sección del manual según su título"""
    titulo_lower = titulo.lower()

    if 'especificacion' in titulo_lower:
        return 'especificaciones'
    elif 'componente' in titulo_lower:
        return 'componentes'
    elif 'procedimiento' in titulo_lower or 'uso' in titulo_lower:
        return 'procedimientos'
    elif 'compatibilidad' in titulo_lower or 'relacion' in titulo_lower:
        return 'compatibilidad'
    elif 'problema' in titulo_lower or 'solucion' in titulo_lower:
        return 'troubleshooting'
    elif 'mantenimiento' in titulo_lower:
        return 'mantenimiento'
    elif 'garantia' in titulo_lower or 'garantía' in titulo_lower:
        return 'garantia'
    else:
        return 'otros'

def load_manuales(base_dir: Path) -> List[Dict]:
    """
    Carga y parsea todos los manuales técnicos
    Busca en:
    - /content/data/manual_*.md
    - /content/data/manuales_productos/manual_*.md
    """
    manuales = []

    # Buscar en múltiples ubicaciones
    search_paths = [
        base_dir / 'manual_*.md',
        base_dir / 'manuales_productos' / 'manual_*.md'
    ]

    md_files = []
    for search_path in search_paths:
        md_files.extend(list(base_dir.glob(str(search_path.relative_to(base_dir)))))

    # También búsqueda recursiva general
    md_files.extend(list(base_dir.rglob('manual_*.md')))

    # Eliminar duplicados
    md_files = list(set(md_files))

    print(f"   📘 Encontrados {len(md_files)} archivos .md")

    for md_file in md_files:
        match = re.search(r'manual_(P\d+)', md_file.name)
        if match:
            id_producto = match.group(1)

            try:
                with open(md_file, 'r', encoding='utf-8') as f:
                    contenido = f.read()

                secciones = parse_manual_sections(contenido)

                for seccion in secciones:
                    manuales.append({
                        'id_producto': id_producto,
                        'tipo_seccion': seccion['tipo'],
                        'titulo': seccion['titulo'],
                        'contenido': fix_encoding(seccion['contenido']),
                        'archivo': md_file.name
                    })
            except Exception as e:
                print(f"      ⚠️ Error leyendo {md_file.name}: {e}")

    return manuales

# ==================== EXTRACCIÓN DE METADATA ====================

def extract_productos_metadata(df: pd.DataFrame) -> Dict:
    """Extrae metadata del dataset de productos"""
    metadata = {
        'total_productos': len(df),
        'columnas': list(df.columns),
        'categorias_unicas': sorted(df['categoria'].dropna().unique().tolist()),
        'subcategorias_unicas': sorted(df['subcategoria'].dropna().unique().tolist()),
        'marcas_unicas': sorted(df['marca'].dropna().unique().tolist()),
        'precio_min': float(df['precio_usd'].min()),
        'precio_max': float(df['precio_usd'].max()),
        'colores_disponibles': sorted(df['color'].dropna().unique().tolist()),
        'voltajes_disponibles': sorted(df['voltaje'].dropna().unique().tolist()),
        'garantia_min': int(df['garantia_meses'].min()),
        'garantia_max': int(df['garantia_meses'].max()),
        'potencia_min': float(df['potencia_w'].min()),
        'potencia_max': float(df['potencia_w'].max()),
        'capacidades_unicas': sorted(df['capacidad'].dropna().unique().tolist())
    }

    return metadata

def extract_ventas_metadata(df: pd.DataFrame) -> Dict:
    """Extrae metadata del dataset de ventas"""
    metadata = {
        'total_ventas': len(df),
        'columnas': list(df.columns),
        'metodos_pago': sorted(df['metodo_pago'].dropna().unique().tolist()),
        'sucursales': sorted(df['sucursal'].dropna().unique().tolist()),
        'provincias': sorted(df['cliente_provincia'].dropna().unique().tolist()),
        'fecha_min': df['fecha'].min(),
        'fecha_max': df['fecha'].max(),
        'total_min': float(df['total'].min()),
        'total_max': float(df['total'].max()),
        'descuento_max': int(df['descuento_pct'].max())
    }

    return metadata

# ==================== CLASE DATA LOADER ====================

class DataLoader:
    """Clase para cargar todos los datos del sistema RAG"""

    def __init__(self, config):
        self.config = config
        self.dataframes = {}
        self.documents = {}
        self.metadata = {}

    def load_all(self):
        """Carga todos los datos del sistema"""
        print("\n" + "="*60)
        print("CARGANDO DATOS DEL SISTEMA RAG")
        print("="*60 + "\n")

        # ========== CARGAR DATAFRAMES CSV ==========
        print("📊 Cargando archivos CSV...")
        for key, filename in self.config.DATA_FILES.items():
            if filename.endswith('.csv'):
                filepath = self.config.DATA_DIR / filename
                if filepath.exists():
                    self.dataframes[key] = load_csv(filepath)
                    print(f"   ✅ {key}: {len(self.dataframes[key])} filas")
                else:
                    print(f"   ⚠️ No encontrado: {filename}")

        # ========== CARGAR FAQs JSON ==========
        print("\n📄 Cargando FAQs...")
        faqs_path = self.config.DATA_DIR / self.config.DATA_FILES['faqs']
        if faqs_path.exists():
            self.documents['faqs'] = load_json(faqs_path)
            print(f"   ✅ FAQs: {len(self.documents['faqs'])} registros")
        else:
            print(f"   ⚠️ No encontrado: {faqs_path.name}")
            self.documents['faqs'] = []

        # ========== CARGAR RESEÑAS ==========
        print("\n📝 Cargando reseñas...")
        self.documents['resenas'] = load_resenas(self.config.DATA_DIR)
        print(f"   ✅ Reseñas cargadas: {len(self.documents['resenas'])} archivos")

        # ========== CARGAR MANUALES ==========
        print("\n📘 Cargando manuales...")
        self.documents['manuales'] = load_manuales(self.config.DATA_DIR)
        print(f"   ✅ Manuales cargados: {len(self.documents['manuales'])} secciones")

        # ========== EXTRAER METADATA ==========
        print("\n📊 Extrayendo metadata...")
        if 'productos' in self.dataframes:
            self.metadata['productos'] = extract_productos_metadata(self.dataframes['productos'])
            print(f"   ✅ Metadata productos:")
            print(f"      - {len(self.metadata['productos']['categorias_unicas'])} categorías")
            print(f"      - {len(self.metadata['productos']['marcas_unicas'])} marcas")
            print(f"      - Precios: ${self.metadata['productos']['precio_min']:.2f} - ${self.metadata['productos']['precio_max']:.2f}")

        if 'ventas' in self.dataframes:
            self.metadata['ventas'] = extract_ventas_metadata(self.dataframes['ventas'])
            print(f"   ✅ Metadata ventas:")
            print(f"      - {self.metadata['ventas']['total_ventas']} transacciones")
            print(f"      - {len(self.metadata['ventas']['sucursales'])} sucursales")
            print(f"      - {len(self.metadata['ventas']['metodos_pago'])} métodos de pago")

        # ========== RESUMEN FINAL ==========
        self._print_summary()

        return self.dataframes, self.documents, self.metadata

    def _print_summary(self):
        """Imprime resumen de datos cargados"""
        print("\n" + "="*60)
        print("RESUMEN DE DATOS CARGADOS")
        print("="*60)

        print(f"\n📊 DataFrames ({len(self.dataframes)}):")
        for key, df in self.dataframes.items():
            print(f"   {key:15s} → {len(df):6d} filas x {len(df.columns):2d} columnas")

        print(f"\n📄 Documentos no tabulares:")
        for key, docs in self.documents.items():
            print(f"   {key:15s} → {len(docs):6d} documentos")

        total_docs = sum(len(docs) for docs in self.documents.values())
        print(f"\n📈 Total documentos para vectorizar: {total_docs}")

        print(f"\n📊 Metadata extraída: {len(self.metadata)} datasets")
        print("="*60 + "\n")

# ==================== EJECUCIÓN ====================

print("📄 Inicializando DataLoader...")
data_loader = DataLoader(config)

dataframes, documents, metadata = data_loader.load_all()

print("✅ Datos cargados exitosamente")
print(f"   Acceso: dataframes, documents, metadata")
print(f"\nEjemplo de uso:")
print(f"   df_productos = dataframes['productos']")
print(f"   faqs = documents['faqs']")
print(f"   meta_productos = metadata['productos']")

📄 Inicializando DataLoader...

CARGANDO DATOS DEL SISTEMA RAG

📊 Cargando archivos CSV...
   ✅ productos: 300 filas
   ✅ vendedores: 100 filas
   ✅ ventas: 10000 filas
   ✅ tickets: 2000 filas
   ✅ inventario: 4100 filas
   ✅ devoluciones: 800 filas

📄 Cargando FAQs...
   ✅ FAQs: 3000 registros

📝 Cargando reseñas...
   📝 Encontrados 2978 archivos .txt
   ✅ Reseñas cargadas: 2978 archivos

📘 Cargando manuales...
   📘 Encontrados 50 archivos .md
   ✅ Manuales cargados: 450 secciones

📊 Extrayendo metadata...
   ✅ Metadata productos:
      - 4 categorías
      - 17 marcas
      - Precios: $28.22 - $2992.33
   ✅ Metadata ventas:
      - 10000 transacciones
      - 23 sucursales
      - 8 métodos de pago

RESUMEN DE DATOS CARGADOS

📊 DataFrames (6):
   productos       →    300 filas x 14 columnas
   vendedores      →    100 filas x 10 columnas
   ventas          →  10000 filas x 15 columnas
   tickets         →   2000 filas x 17 columnas
   inventario      →   4100 filas x 14 columnas
   d

In [4]:
"""
============================================
CELDA 4: BASE DE DATOS VECTORIAL CON CHROMADB
============================================
Implementa los 4 puntos requeridos:
1. ChromaDB como base de datos vectorial
2. Modelo de embeddings multilenguaje
3. Text Splitter con parámetros configurables
4. Búsqueda híbrida (Semántica + BM25) + ReRank
"""

import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer, CrossEncoder
from rank_bm25 import BM25Okapi
import numpy as np
from typing import List, Dict, Tuple, Optional
import re
import pandas as pd

print("\n" + "="*60)
print("CONSTRUYENDO BASE DE DATOS VECTORIAL")
print("="*60 + "\n")

# ==================== PUNTO 1: TEXT SPLITTER ====================

class DocumentProcessor:
    """
    PUNTO 3: TEXT SPLITTER

    Procesa documentos y los divide en chunks con parámetros configurables:
    - chunk_size: Tamaño máximo de cada chunk
    - chunk_overlap: Solapamiento entre chunks
    - separators: Lista jerárquica de separadores
    """

    def __init__(self, config):
        self.config = config
        self.chunk_size = config.SPLITTER_CONFIG['chunk_size']
        self.chunk_overlap = config.SPLITTER_CONFIG['chunk_overlap']
        self.separators = config.SPLITTER_CONFIG['separators']

        print(f"📝 Text Splitter configurado:")
        print(f"   - Chunk size: {self.chunk_size}")
        print(f"   - Chunk overlap: {self.chunk_overlap}")
        print(f"   - Separadores: {self.separators}")

    def split_text(self, text: str) -> List[str]:
        """Divide texto en chunks con overlap usando separadores jerárquicos"""
        chunks = []

        # Intentar dividir por separadores jerárquicos
        for separator in self.separators:
            if separator in text:
                parts = text.split(separator)
                current_chunk = ""

                for part in parts:
                    if len(current_chunk) + len(part) < self.chunk_size:
                        current_chunk += part + separator
                    else:
                        if current_chunk:
                            chunks.append(current_chunk.strip())
                        current_chunk = part + separator

                if current_chunk:
                    chunks.append(current_chunk.strip())

                if chunks:
                    return chunks

        # Si no hay separadores, dividir por caracteres con overlap
        for i in range(0, len(text), self.chunk_size - self.chunk_overlap):
            chunk = text[i:i + self.chunk_size]
            if chunk.strip():
                chunks.append(chunk.strip())

        return chunks if chunks else [text]

    def prepare_faqs(self, faqs: List[Dict]) -> List[Dict]:
        """Prepara FAQs con contexto mejorado"""
        documents = []

        for faq in faqs:
            # Agregar prefijo según categoría para mejor contexto
            categoria = faq.get('categoria', '')

            if categoria == 'Uso':
                prefix = "INSTRUCCIONES DE USO - "
            elif categoria == 'Problemas Comunes':
                prefix = "SOLUCIÓN DE PROBLEMAS - "
            elif categoria == 'Mantenimiento':
                prefix = "MANTENIMIENTO - "
            elif categoria == 'Garantía':
                prefix = "GARANTÍA - "
            elif categoria == 'Especificaciones':
                prefix = "ESPECIFICACIONES TÉCNICAS - "
            else:
                prefix = ""

            text = f"{prefix}Pregunta: {faq['pregunta']}\nRespuesta: {faq['respuesta']}"

            documents.append({
                'id': faq['id_faq'],
                'text': text,
                'metadata': {
                    'source': 'faq',
                    'id_producto': faq.get('id_producto', ''),
                    'nombre_producto': faq.get('nombre_producto', ''),
                    'categoria': faq.get('categoria', ''),
                    'pregunta': faq['pregunta']
                }
            })

        return documents

    def prepare_manuales(self, manuales: List[Dict]) -> List[Dict]:
        """Prepara manuales divididos en chunks con contexto"""
        documents = []

        for idx, manual in enumerate(manuales):
            chunks = self.split_text(manual['contenido'])

            for chunk_idx, chunk in enumerate(chunks):
                doc_id = f"{manual['id_producto']}_{manual['tipo_seccion']}_{idx}_{chunk_idx}"

                # Agregar prefijos contextuales según tipo
                tipo = manual['tipo_seccion']

                if tipo == 'procedimientos':
                    prefix = "INSTRUCCIONES DE USO PASO A PASO - "
                elif tipo == 'troubleshooting':
                    prefix = "SOLUCIÓN DE PROBLEMAS - "
                elif tipo == 'especificaciones':
                    prefix = "ESPECIFICACIONES TÉCNICAS - "
                elif tipo == 'mantenimiento':
                    prefix = "MANTENIMIENTO - "
                elif tipo == 'compatibilidad':
                    prefix = "COMPATIBILIDAD Y ACCESORIOS - "
                else:
                    prefix = ""

                text = f"{prefix}{manual['titulo']}\n\n{chunk}"

                documents.append({
                    'id': doc_id,
                    'text': text,
                    'metadata': {
                        'source': 'manual',
                        'id_producto': manual['id_producto'],
                        'tipo_seccion': manual['tipo_seccion'],
                        'titulo': manual['titulo'],
                        'archivo': manual.get('archivo', '')
                    }
                })

        return documents

    def prepare_resenas(self, resenas: List[Dict]) -> List[Dict]:
        """Prepara reseñas con contexto de opinión"""
        documents = []

        for resena in resenas:
            text = f"OPINIÓN DE USUARIO - Reseña:\n{resena['texto']}"

            documents.append({
                'id': resena['id_resena'],
                'text': text,
                'metadata': {
                    'source': 'resena',
                    'id_resena': resena['id_resena'],
                    'archivo': resena.get('archivo', '')
                }
            })

        return documents

# ==================== PUNTO 1: CHROMADB + PUNTO 2: EMBEDDINGS ====================

class VectorDatabase:
    """
    PUNTO 1: ChromaDB como base de datos vectorial
    PUNTO 2: Modelo de embeddings multilenguaje

    Gestiona la base de datos vectorial con:
    - ChromaDB para almacenamiento persistente
    - Sentence Transformers para embeddings en español
    - Cross-Encoder para reranking
    """

    def __init__(self, config):
        self.config = config

        # Inicializar ChromaDB
        print("💾 Inicializando ChromaDB...")
        self.client = chromadb.Client(Settings(
            persist_directory=config.CHROMA_CONFIG['persist_directory'],
            anonymized_telemetry=False
        ))

        # PUNTO 2: Cargar modelo de embeddings multilenguaje
        print(f"\n🧠 Cargando modelo de embeddings...")
        print(f"   Modelo: {config.EMBEDDING_CONFIG['model_name']}")
        print(f"   Dispositivo: {config.EMBEDDING_CONFIG['device']}")
        print(f"   Normalización: {config.EMBEDDING_CONFIG['normalize_embeddings']}")

        self.embedding_model = SentenceTransformer(
            config.EMBEDDING_CONFIG['model_name'],
            device=config.EMBEDDING_CONFIG['device']
        )
        print(f"✅ Modelo de embeddings cargado")

        # Cargar modelo de reranking
        print(f"\n🔄 Cargando modelo de reranking...")
        self.reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
        print(f"✅ Modelo de reranking cargado")

        # Eliminar colección existente si existe
        try:
            self.client.delete_collection(name=config.CHROMA_CONFIG['collection_name'])
            print("\n🗑️ Colección anterior eliminada")
        except:
            pass

        # Crear nueva colección
        print(f"\n📦 Creando colección: {config.CHROMA_CONFIG['collection_name']}")
        self.collection = self.client.create_collection(
            name=config.CHROMA_CONFIG['collection_name'],
            metadata={"hnsw:space": config.CHROMA_CONFIG['distance_metric']}
        )
        print("✅ Colección creada")

    def add_documents(self, documents: List[Dict]):
        """Agrega documentos a la colección con sus embeddings"""
        if not documents:
            return

        print(f"\n📝 Agregando {len(documents)} documentos...")

        # Preparar datos para ChromaDB
        ids = [doc['id'] for doc in documents]
        texts = [doc['text'] for doc in documents]
        metadatas = [doc['metadata'] for doc in documents]

        # PUNTO 2: Generar embeddings con el modelo multilenguaje
        print("🧠 Generando embeddings...")
        embeddings = self.embedding_model.encode(
            texts,
            show_progress_bar=True,
            normalize_embeddings=self.config.EMBEDDING_CONFIG['normalize_embeddings']
        ).tolist()

        # Agregar a ChromaDB en lotes
        batch_size = 100
        for i in range(0, len(ids), batch_size):
            batch_ids = ids[i:i+batch_size]
            batch_embeddings = embeddings[i:i+batch_size]
            batch_texts = texts[i:i+batch_size]
            batch_metadatas = metadatas[i:i+batch_size]

            self.collection.add(
                ids=batch_ids,
                embeddings=batch_embeddings,
                documents=batch_texts,
                metadatas=batch_metadatas
            )

        print(f"✅ {len(documents)} documentos agregados a ChromaDB")

    def search(self, query: str, top_k: int = 10, filters: Optional[Dict] = None) -> List[Dict]:
        """Búsqueda semántica en ChromaDB"""
        # Generar embedding de la query
        query_embedding = self.embedding_model.encode(
            [query],
            normalize_embeddings=self.config.EMBEDDING_CONFIG['normalize_embeddings']
        ).tolist()[0]

        # Preparar filtros (where clause)
        where_clause = None
        if filters:
            where_clause = filters

        # Buscar en ChromaDB
        results = self.collection.query(
            query_embeddings=[query_embedding],
            n_results=top_k,
            where=where_clause
        )

        # Formatear resultados
        formatted_results = []
        for i in range(len(results['ids'][0])):
            formatted_results.append({
                'id': results['ids'][0][i],
                'text': results['documents'][0][i],
                'metadata': results['metadatas'][0][i],
                'distance': results['distances'][0][i],
                'score': 1 - results['distances'][0][i]
            })

        return formatted_results

# ==================== PUNTO 4: BÚSQUEDA HÍBRIDA (BM25) ====================

class BM25Search:
    """
    PUNTO 4: Búsqueda por keywords con BM25

    Complementa la búsqueda semántica con búsqueda léxica
    """

    def __init__(self):
        self.corpus = []
        self.documents = []
        self.bm25 = None
        print("🔍 Inicializando BM25 para búsqueda híbrida...")

    def tokenize(self, text: str) -> List[str]:
        """Tokeniza texto en palabras"""
        text = text.lower()
        tokens = re.findall(r'\b\w+\b', text)
        return tokens

    def index_documents(self, documents: List[Dict]):
        """Indexa documentos para BM25"""
        print(f"📚 Indexando {len(documents)} documentos para BM25...")

        self.documents = documents
        self.corpus = [self.tokenize(doc['text']) for doc in documents]
        self.bm25 = BM25Okapi(self.corpus)

        print("✅ BM25 indexado")

    def search(self, query: str, top_k: int = 10) -> List[Dict]:
        """Búsqueda con BM25"""
        if not self.bm25:
            return []

        tokenized_query = self.tokenize(query)
        scores = self.bm25.get_scores(tokenized_query)

        # Obtener top-k
        top_indices = np.argsort(scores)[::-1][:top_k]

        results = []
        for idx in top_indices:
            if scores[idx] > 0:
                results.append({
                    'id': self.documents[idx]['id'],
                    'text': self.documents[idx]['text'],
                    'metadata': self.documents[idx]['metadata'],
                    'score': float(scores[idx])
                })

        return results

# ==================== PUNTO 4: BÚSQUEDA HÍBRIDA + RERANK ====================

class HybridSearch:
    """
    PUNTO 4: Búsqueda híbrida (Semántica + BM25) + ReRank

    Combina:
    1. Búsqueda semántica (ChromaDB + embeddings)
    2. Búsqueda léxica (BM25)
    3. Fusión de resultados (RRF)
    4. Reranking con Cross-Encoder
    """

    def __init__(self, vector_db: VectorDatabase, bm25_search: BM25Search, config):
        self.vector_db = vector_db
        self.bm25_search = bm25_search
        self.config = config

        print("\n🔀 Búsqueda híbrida configurada:")
        print(f"   - Top-K semántico: {config.HYBRID_SEARCH_CONFIG['semantic_top_k']}")
        print(f"   - Top-K BM25: {config.HYBRID_SEARCH_CONFIG['bm25_top_k']}")
        print(f"   - Top-K final (rerank): {config.HYBRID_SEARCH_CONFIG['rerank_top_k']}")
        print(f"   - Alpha (peso semántico): {config.HYBRID_SEARCH_CONFIG['alpha']}")

    def search(self, query: str, top_k: int = 5, filters: Optional[Dict] = None, auto_filter: bool = True) -> List[Dict]:
        """
        Búsqueda híbrida completa con 4 pasos:
        1. Búsqueda semántica
        2. Búsqueda BM25
        3. Fusión RRF
        4. Reranking
        """

        # Filtrado inteligente automático
        if auto_filter and filters is None:
            filters = self._smart_filter(query)

        # 1. Búsqueda semántica (70% peso)
        semantic_results = self.vector_db.search(
            query,
            top_k=self.config.HYBRID_SEARCH_CONFIG['semantic_top_k'],
            filters=filters
        )

        # 2. Búsqueda BM25 (30% peso)
        bm25_results = self.bm25_search.search(
            query,
            top_k=self.config.HYBRID_SEARCH_CONFIG['bm25_top_k']
        )

        # 3. Fusión con RRF
        fused_results = self._reciprocal_rank_fusion(
            semantic_results,
            bm25_results,
            alpha=self.config.HYBRID_SEARCH_CONFIG['alpha']
        )

        # 4. Reranking con Cross-Encoder
        reranked_results = self._rerank(query, fused_results, top_k)

        return reranked_results

    def _smart_filter(self, query: str) -> Optional[Dict]:
        """Detecta tipo de pregunta y aplica filtros apropiados"""
        query_lower = query.lower()

        # Para "cómo usar" -> SOLO manuales y FAQs de uso
        if any(word in query_lower for word in ['cómo', 'como', 'usar', 'utilizar', 'hacer', 'preparar']):
            return {"source": {"$in": ["manual", "faq"]}}

        # Para "opinión" -> SOLO reseñas
        if any(word in query_lower for word in ['opinión', 'opinion', 'reseña', 'comentario', 'qué dicen']):
            return {"source": {"$in": ["resena"]}}

        # Para "problema" -> tickets, FAQs y manuales
        if any(word in query_lower for word in ['problema', 'no funciona', 'error', 'falla']):
            return {"source": {"$in": ["ticket", "faq", "manual"]}}

        # Sin filtro para preguntas generales
        return None

    def _reciprocal_rank_fusion(self, semantic_results: List[Dict], bm25_results: List[Dict], k: int = 60, alpha: float = 0.7) -> List[Dict]:
        """Fusiona resultados usando RRF con pesos ajustables"""
        scores = {}
        docs = {}

        # Procesar resultados semánticos (70%)
        for rank, result in enumerate(semantic_results):
            doc_id = result['id']
            scores[doc_id] = scores.get(doc_id, 0) + alpha * (1 / (k + rank + 1))
            docs[doc_id] = result

        # Procesar resultados BM25 (30%)
        for rank, result in enumerate(bm25_results):
            doc_id = result['id']
            scores[doc_id] = scores.get(doc_id, 0) + (1 - alpha) * (1 / (k + rank + 1))
            if doc_id not in docs:
                docs[doc_id] = result

        # Ordenar por score fusionado
        sorted_ids = sorted(scores.keys(), key=lambda x: scores[x], reverse=True)

        fused_results = []
        for doc_id in sorted_ids:
            result = docs[doc_id].copy()
            result['fusion_score'] = scores[doc_id]
            fused_results.append(result)

        return fused_results

    def _rerank(self, query: str, results: List[Dict], top_k: int) -> List[Dict]:
        """Reordena resultados con cross-encoder"""
        if not results:
            return []

        # Preparar pares query-documento
        pairs = [[query, result['text']] for result in results]

        # Calcular scores con cross-encoder
        rerank_scores = self.vector_db.reranker.predict(pairs)

        # Agregar scores y reordenar
        for i, result in enumerate(results):
            result['rerank_score'] = float(rerank_scores[i])

        # Ordenar por rerank_score
        reranked = sorted(results, key=lambda x: x['rerank_score'], reverse=True)

        return reranked[:top_k]

# ==================== CONSTRUCCIÓN DEL SISTEMA ====================

print("\n" + "="*60)
print("PROCESANDO Y VECTORIZANDO DOCUMENTOS")
print("="*60 + "\n")

# 1. Inicializar procesador de documentos
processor = DocumentProcessor(config)

# 2. Preparar todos los documentos
print("\n📄 Preparando documentos...")
all_documents = []

all_documents.extend(processor.prepare_faqs(documents['faqs']))
print(f"✅ FAQs preparados: {len(documents['faqs'])}")

all_documents.extend(processor.prepare_manuales(documents['manuales']))
print(f"✅ Manuales preparados: {len(documents['manuales'])}")

all_documents.extend(processor.prepare_resenas(documents['resenas']))
print(f"✅ Reseñas preparadas: {len(documents['resenas'])}")

print(f"\n📊 Total documentos a indexar: {len(all_documents)}")

# 3. Inicializar bases de datos
vector_db = VectorDatabase(config)
bm25_search = BM25Search()

# 4. Indexar documentos
vector_db.add_documents(all_documents)
bm25_search.index_documents(all_documents)

# 5. Crear búsqueda híbrida
hybrid_search = HybridSearch(vector_db, bm25_search, config)

print("\n" + "="*60)
print("✅ BASE DE DATOS VECTORIAL COMPLETA")
print("="*60)
print(f"""
✅ PUNTO 1: ChromaDB configurado
   - Colección: {config.CHROMA_CONFIG['collection_name']}
   - Documentos: {len(all_documents)}
   - Distancia: {config.CHROMA_CONFIG['distance_metric']}

✅ PUNTO 2: Embeddings multilenguaje
   - Modelo: {config.EMBEDDING_CONFIG['model_name']}
   - Normalización: {config.EMBEDDING_CONFIG['normalize_embeddings']}

✅ PUNTO 3: Text Splitter configurado
   - Chunk size: {config.SPLITTER_CONFIG['chunk_size']}
   - Chunk overlap: {config.SPLITTER_CONFIG['chunk_overlap']}

✅ PUNTO 4: Búsqueda híbrida + ReRank
   - Semántica (ChromaDB) + BM25
   - Fusión RRF (alpha={config.HYBRID_SEARCH_CONFIG['alpha']})
   - ReRank con Cross-Encoder

🔍 Interfaz de búsqueda:
   hybrid_search.search(query, top_k=5)
""")
print("="*60 + "\n")


CONSTRUYENDO BASE DE DATOS VECTORIAL


PROCESANDO Y VECTORIZANDO DOCUMENTOS

📝 Text Splitter configurado:
   - Chunk size: 400
   - Chunk overlap: 50
   - Separadores: ['\n\n', '\n', '. ', ' ', '']

📄 Preparando documentos...
✅ FAQs preparados: 3000
✅ Manuales preparados: 450
✅ Reseñas preparadas: 2978

📊 Total documentos a indexar: 7122
💾 Inicializando ChromaDB...

🧠 Cargando modelo de embeddings...
   Modelo: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
   Dispositivo: cpu
   Normalización: True


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✅ Modelo de reranking cargado

📦 Creando colección: electrodomesticos_docs
✅ Colección creada
🔍 Inicializando BM25 para búsqueda híbrida...

📝 Agregando 7122 documentos...
🧠 Generando embeddings...


Batches:   0%|          | 0/223 [00:00<?, ?it/s]

✅ 7122 documentos agregados a ChromaDB
📚 Indexando 7122 documentos para BM25...
✅ BM25 indexado

🔀 Búsqueda híbrida configurada:
   - Top-K semántico: 20
   - Top-K BM25: 20
   - Top-K final (rerank): 5
   - Alpha (peso semántico): 0.7

✅ BASE DE DATOS VECTORIAL COMPLETA

✅ PUNTO 1: ChromaDB configurado
   - Colección: electrodomesticos_docs
   - Documentos: 7122
   - Distancia: cosine

✅ PUNTO 2: Embeddings multilenguaje
   - Modelo: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
   - Normalización: True

✅ PUNTO 3: Text Splitter configurado
   - Chunk size: 400
   - Chunk overlap: 50

✅ PUNTO 4: Búsqueda híbrida + ReRank
   - Semántica (ChromaDB) + BM25
   - Fusión RRF (alpha=0.7)
   - ReRank con Cross-Encoder

🔍 Interfaz de búsqueda:
   hybrid_search.search(query, top_k=5)




In [5]:
"""
============================================
CELDA 5: PRUEBAS DE LA BASE VECTORIAL
============================================
Pruebas para verificar el correcto funcionamiento
"""

print("\n" + "="*60)
print("PRUEBAS DE LA BASE DE DATOS VECTORIAL")
print("="*60 + "\n")

# ==================== PRUEBA 1: BÚSQUEDA SIMPLE ====================

print("📝 PRUEBA 1: Búsqueda simple")
print("-" * 60)

test_query = "¿Cómo uso mi licuadora para hacer smoothies?"
print(f"Query: '{test_query}'\n")

results = hybrid_search.search(test_query, top_k=3)

print(f"Resultados encontrados: {len(results)}\n")

for i, result in enumerate(results, 1):
    print(f"{i}. [{result['metadata']['source'].upper()}]")
    print(f"   Score: {result['rerank_score']:.4f}")
    print(f"   Texto: {result['text'][:150]}...")
    if 'id_producto' in result['metadata']:
        print(f"   Producto: {result['metadata'].get('id_producto', 'N/A')}")
    print()

# ==================== PRUEBA 2: DIFERENTES TIPOS DE CONSULTAS ====================

print("\n" + "="*60)
print("📝 PRUEBA 2: Diferentes tipos de consultas")
print("="*60 + "\n")

test_queries = [
    "¿Qué opinan los usuarios de las licuadoras?",
    "Mi licuadora no enciende, ¿qué hago?",
    "¿Cómo se limpia el filtro de la cafetera?",
]

for test_query in test_queries:
    print(f"Query: '{test_query}'")
    results = hybrid_search.search(test_query, top_k=2)
    print(f"Resultados: {len(results)}")

    if results:
        top_result = results[0]
        print(f"  → Top resultado: [{top_result['metadata']['source'].upper()}]")
        print(f"    Score: {top_result['rerank_score']:.4f}")
        print(f"    Preview: {top_result['text'][:100]}...")
    print()

# ==================== PRUEBA 3: VERIFICAR FILTROS AUTOMÁTICOS ====================

print("\n" + "="*60)
print("📝 PRUEBA 3: Filtros automáticos por tipo de pregunta")
print("="*60 + "\n")

# Test filtros automáticos
test_cases = [
    ("¿Cómo usar mi procesadora?", "manual/faq"),
    ("¿Qué opinan de este producto?", "resena"),
    ("Mi producto tiene un problema", "ticket/faq/manual")
]

for query, expected_source in test_cases:
    print(f"Query: '{query}'")
    print(f"Fuente esperada: {expected_source}")

    results = hybrid_search.search(query, top_k=3)
    if results:
        actual_sources = [r['metadata']['source'] for r in results]
        print(f"Fuentes obtenidas: {set(actual_sources)}")
    print()

# ==================== PRUEBA 4: ESTADÍSTICAS DEL SISTEMA ====================

print("\n" + "="*60)
print("📊 ESTADÍSTICAS DEL SISTEMA")
print("="*60 + "\n")

# Contar documentos por fuente
sources_count = {}
for doc in all_documents:
    source = doc['metadata']['source']
    sources_count[source] = sources_count.get(source, 0) + 1

print("Documentos por fuente:")
for source, count in sorted(sources_count.items()):
    print(f"  {source:10s}: {count:4d} documentos")

print(f"\nTotal: {len(all_documents)} documentos indexados")

# Información del modelo
print(f"\n🧠 Modelo de embeddings:")
print(f"  Nombre: {config.EMBEDDING_CONFIG['model_name']}")
print(f"  Dimensión: {vector_db.embedding_model.get_sentence_embedding_dimension()}")

print("\n" + "="*60)
print("✅ TODAS LAS PRUEBAS COMPLETADAS")
print("="*60 + "\n")

# ==================== FUNCIÓN DE BÚSQUEDA INTERACTIVA ====================

def buscar(query: str, top_k: int = 5, mostrar_texto: bool = True):
    """
    Función helper para hacer búsquedas rápidas

    Args:
        query: Consulta en lenguaje natural
        top_k: Número de resultados a devolver
        mostrar_texto: Si True, muestra el texto completo
    """
    print(f"\n🔍 Buscando: '{query}'")
    print("-" * 60)

    results = hybrid_search.search(query, top_k=top_k)

    print(f"\n📊 Encontrados {len(results)} resultados:\n")

    for i, result in enumerate(results, 1):
        print(f"{i}. [{result['metadata']['source'].upper()}] Score: {result['rerank_score']:.4f}")

        if 'id_producto' in result['metadata'] and result['metadata']['id_producto']:
            print(f"   Producto: {result['metadata']['id_producto']}")

        if mostrar_texto:
            text = result['text']
            if len(text) > 200:
                text = text[:200] + "..."
            print(f"   {text}")

        print()

    return results

print("✅ Función de búsqueda lista: buscar('tu consulta', top_k=5)")
print("\nEjemplo de uso:")
print("  results = buscar('¿Cómo hacer smoothies?', top_k=3)")


PRUEBAS DE LA BASE DE DATOS VECTORIAL

📝 PRUEBA 1: Búsqueda simple
------------------------------------------------------------
Query: '¿Cómo uso mi licuadora para hacer smoothies?'

Resultados encontrados: 3

1. [FAQ]
   Score: 1.6231
   Texto: INSTRUCCIONES DE USO - Pregunta: ¿Cómo se usa correctamente este producto?
Respuesta: El Licuadora de TechHome está diseñado para uso doméstico. Revis...
   Producto: P0001

2. [FAQ]
   Score: 1.6231
   Texto: INSTRUCCIONES DE USO - Pregunta: ¿Cómo se usa correctamente este producto?
Respuesta: El Licuadora de TechHome está diseñado para uso doméstico. Revis...
   Producto: P0001

3. [FAQ]
   Score: 1.6014
   Texto: INSTRUCCIONES DE USO - Pregunta: ¿Cómo se usa correctamente este producto?
Respuesta: El Compacto Licuadora de ChefMaster está diseñado para uso domés...
   Producto: P0004


📝 PRUEBA 2: Diferentes tipos de consultas

Query: '¿Qué opinan los usuarios de las licuadoras?'
Resultados: 2
  → Top resultado: [RESENA]
    Score: 3.1803
  

In [6]:
"""
============================================
CELDA 6: BASE DE DATOS TABULAR
============================================
Implementa consultas dinámicas a datos tabulares usando LLM (GROQ)
"""

import pandas as pd
import json
from typing import Dict, List, Optional, Any
from groq import Groq

print("\n" + "="*60)
print("CONFIGURANDO BASE DE DATOS TABULAR")
print("="*60 + "\n")

# ==================== GENERADOR DE FILTROS CON LLM ====================

class TableFilterGenerator:
    """
    Genera filtros dinámicos para pandas usando GROQ
    Convierte lenguaje natural a expresiones de filtrado
    """

    def __init__(self, api_key: str, metadata: Dict):
        self.client = Groq(api_key=api_key)
        self.metadata = metadata

        print("🤖 Generador de filtros inicializado")
        print(f"   Modelo: llama-3.3-70b-versatile")

    def generate_filter(self, query: str, table_name: str) -> Dict[str, Any]:
        """
        Genera filtros a partir de lenguaje natural

        Args:
            query: Consulta en lenguaje natural
            table_name: Nombre de la tabla (productos, ventas, etc)

        Returns:
            Dict con filtros y operaciones
        """

        # Obtener metadata de la tabla
        table_metadata = self.metadata.get(table_name, {})

        # Construir prompt con información de la tabla
        prompt = self._build_filter_prompt(query, table_name, table_metadata)

        try:
            response = self.client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=[
                    {
                        "role": "system",
                        "content": """Eres un experto en generar filtros para pandas DataFrames.

IMPORTANTE:
- Responde SOLO con un objeto JSON válido
- NO uses markdown (```json)
- NO agregues explicaciones
- Para búsquedas de texto usa "contains" (no ==)
- Para rangos de precio/números usa <, >, <=, >=

Estructura del JSON:
{
    "filters": [
        {"column": "nombre_columna", "operator": "==|>|<|>=|<=|contains|in", "value": valor}
    ],
    "sort_by": "columna",
    "ascending": true,
    "limit": 20
}

EJEMPLOS:
- "licuadoras de menos de $200" → {"filters": [{"column": "nombre", "operator": "contains", "value": "Licuadora"}, {"column": "precio_usd", "operator": "<", "value": 200}]}
- "productos TechHome" → {"filters": [{"column": "marca", "operator": "==", "value": "TechHome"}]}
- "garantía mayor a 24 meses" → {"filters": [{"column": "garantia_meses", "operator": ">", "value": 24}]}

IMPORTANTE para tabla 'productos':
- Para buscar tipo de producto (licuadora, cafetera, etc) usar columna "nombre" o "subcategoria"
- La columna "categoria" solo tiene: Cocina, Climatización, Lavado, Audio y Video"""
                    },
                    {
                        "role": "user",
                        "content": prompt
                    }
                ],
                temperature=0.1,
                max_tokens=500
            )

            # Extraer respuesta
            result = response.choices[0].message.content.strip()

            # Limpiar markdown si existe
            result = result.replace('```json', '').replace('```', '').strip()

            # Parsear JSON
            filters = json.loads(result)

            return filters

        except Exception as e:
            print(f"⚠️ Error generando filtros: {e}")
            return {"filters": []}

    def _build_filter_prompt(self, query: str, table_name: str, metadata: Dict) -> str:
        """Construye el prompt con información de la tabla"""

        prompt = f"""Tabla: {table_name}

Información de la tabla:
"""

        if 'columnas' in metadata:
            prompt += f"Columnas disponibles: {', '.join(metadata['columnas'])}\n"

        if table_name == 'productos':
            prompt += f"""
Valores únicos importantes:
- Categorías: {metadata.get('categorias_unicas', [])}
- Marcas: {metadata.get('marcas_unicas', [])}
- Colores: {metadata.get('colores_disponibles', [])}
- Voltajes: {metadata.get('voltajes_disponibles', [])}
- Rango de precios: ${metadata.get('precio_min', 0):.2f} - ${metadata.get('precio_max', 0):.2f}
- Rango de garantía: {metadata.get('garantia_min', 0)} - {metadata.get('garantia_max', 0)} meses
- Rango de potencia: {metadata.get('potencia_min', 0)} - {metadata.get('potencia_max', 0)} W
"""

        elif table_name == 'ventas':
            prompt += f"""
Valores únicos importantes:
- Métodos de pago: {metadata.get('metodos_pago', [])}
- Sucursales: {metadata.get('sucursales', [])}
- Provincias: {metadata.get('provincias', [])}
- Rango de fechas: {metadata.get('fecha_min', '')} - {metadata.get('fecha_max', '')}
- Rango de totales: ${metadata.get('total_min', 0):.2f} - ${metadata.get('total_max', 0):.2f}
"""

        prompt += f"""
Consulta del usuario: "{query}"

Genera los filtros apropiados en formato JSON.
Operadores disponibles: ==, >, <, >=, <=, contains, in
"""

        return prompt

# ==================== BUSCADOR EN TABLAS ====================

class TableSearcher:
    """
    Realiza búsquedas en DataFrames usando filtros generados por LLM
    """

    def __init__(self, dataframes: Dict[str, pd.DataFrame], filter_generator: TableFilterGenerator):
        self.dataframes = dataframes
        self.filter_generator = filter_generator

        print(f"\n📊 Buscador de tablas inicializado")
        print(f"   Tablas disponibles: {list(dataframes.keys())}")

    def search(self, query: str, table_name: str, max_results: int = 10) -> pd.DataFrame:
        """
        Busca en una tabla usando lenguaje natural

        Args:
            query: Consulta en lenguaje natural
            table_name: Nombre de la tabla
            max_results: Número máximo de resultados

        Returns:
            DataFrame con resultados filtrados
        """

        if table_name not in self.dataframes:
            print(f"⚠️ Tabla '{table_name}' no encontrada")
            return pd.DataFrame()

        df = self.dataframes[table_name].copy()

        # Generar filtros con LLM
        filter_spec = self.filter_generator.generate_filter(query, table_name)

        # Debug: mostrar filtros generados
        print(f"🔍 Filtros generados: {json.dumps(filter_spec, indent=2, ensure_ascii=False)}")

        # Aplicar filtros
        df_filtered = self._apply_filters(df, filter_spec)

        # Aplicar ordenamiento si existe
        if 'sort_by' in filter_spec and filter_spec['sort_by']:
            ascending = filter_spec.get('ascending', True)
            if filter_spec['sort_by'] in df_filtered.columns:
                df_filtered = df_filtered.sort_values(
                    by=filter_spec['sort_by'],
                    ascending=ascending
                )

        # Aplicar límite
        limit = filter_spec.get('limit', max_results)
        df_filtered = df_filtered.head(limit)

        return df_filtered

    def _apply_filters(self, df: pd.DataFrame, filter_spec: Dict) -> pd.DataFrame:
        """Aplica los filtros al DataFrame"""

        if 'filters' not in filter_spec or not filter_spec['filters']:
            return df

        mask = pd.Series([True] * len(df))

        for f in filter_spec['filters']:
            column = f.get('column')
            operator = f.get('operator')
            value = f.get('value')

            if column not in df.columns:
                continue

            try:
                if operator == '==':
                    mask &= (df[column] == value)
                elif operator == '>':
                    mask &= (df[column] > value)
                elif operator == '<':
                    mask &= (df[column] < value)
                elif operator == '>=':
                    mask &= (df[column] >= value)
                elif operator == '<=':
                    mask &= (df[column] <= value)
                elif operator == 'contains':
                    mask &= df[column].astype(str).str.contains(str(value), case=False, na=False)
                elif operator == 'in':
                    if isinstance(value, list):
                        mask &= df[column].isin(value)
            except Exception as e:
                print(f"⚠️ Error aplicando filtro en {column}: {e}")
                continue

        return df[mask]

    def get_summary(self, table_name: str) -> str:
        """Obtiene un resumen de la tabla"""

        if table_name not in self.dataframes:
            return f"Tabla '{table_name}' no encontrada"

        df = self.dataframes[table_name]

        summary = f"""Tabla: {table_name}
Filas: {len(df)}
Columnas: {len(df.columns)}
Columnas disponibles: {', '.join(df.columns.tolist())}
"""

        return summary

# ==================== INICIALIZACIÓN ====================

print("🔧 Inicializando componentes de base de datos tabular...\n")

# Crear generador de filtros
filter_generator = TableFilterGenerator(
    api_key=config.GROQ_API_KEY,
    metadata=metadata
)

# Crear buscador de tablas
table_searcher = TableSearcher(
    dataframes=dataframes,
    filter_generator=filter_generator
)

print("\n" + "="*60)
print("✅ BASE DE DATOS TABULAR LISTA")
print("="*60)
print("""
Interfaz de búsqueda:
    results = table_searcher.search(
        query="¿Cuáles son las licuadoras de menos de $200?",
        table_name='productos',
        max_results=10
    )

Tablas disponibles:
""")

for table_name in dataframes.keys():
    df = dataframes[table_name]
    print(f"    - {table_name}: {len(df)} filas, {len(df.columns)} columnas")

print("="*60 + "\n")


CONFIGURANDO BASE DE DATOS TABULAR

🔧 Inicializando componentes de base de datos tabular...

🤖 Generador de filtros inicializado
   Modelo: llama-3.3-70b-versatile

📊 Buscador de tablas inicializado
   Tablas disponibles: ['productos', 'vendedores', 'ventas', 'tickets', 'inventario', 'devoluciones']

✅ BASE DE DATOS TABULAR LISTA

Interfaz de búsqueda:
    results = table_searcher.search(
        query="¿Cuáles son las licuadoras de menos de $200?",
        table_name='productos',
        max_results=10
    )

Tablas disponibles:

    - productos: 300 filas, 14 columnas
    - vendedores: 100 filas, 10 columnas
    - ventas: 10000 filas, 15 columnas
    - tickets: 2000 filas, 17 columnas
    - inventario: 4100 filas, 14 columnas
    - devoluciones: 800 filas, 14 columnas



In [11]:
"""
============================================
CELDA 7: PRUEBAS DE BASE TABULAR
============================================
"""

print("\n" + "="*60)
print("PRUEBAS DE LA BASE DE DATOS TABULAR")
print("="*60 + "\n")

# ==================== PRUEBA 1: FILTRO POR PRECIO ====================

print("📝 PRUEBA 1: Filtro por precio")
print("-" * 60)

query = "¿Cuáles son las licuadoras de menos de $300?"
print(f"Query: '{query}'\n")

results = table_searcher.search(query, 'productos', max_results=5)

print(f"Resultados encontrados: {len(results)}\n")
if len(results) > 0:
    print(results[['id_producto', 'nombre', 'marca', 'precio_usd', 'categoria']].to_string(index=False))
else:
    print("No se encontraron resultados")

# ==================== PRUEBA 2: FILTRO POR MARCA ====================

print("\n\n" + "="*60)
print("📝 PRUEBA 2: Filtro por marca")
print("-" * 60)

query = "Productos de la marca TechHome"
print(f"Query: '{query}'\n")

results = table_searcher.search(query, 'productos', max_results=5)

print(f"Resultados encontrados: {len(results)}\n")
if len(results) > 0:
    print(results[['id_producto', 'nombre', 'marca', 'precio_usd']].to_string(index=False))
else:
    print("No se encontraron resultados")

# ==================== PRUEBA 3: FILTRO POR GARANTÍA ====================

print("\n\n" + "="*60)
print("📝 PRUEBA 3: Filtro por garantía")
print("-" * 60)

query = "Productos con garantía mayor a 24 meses"
print(f"Query: '{query}'\n")

results = table_searcher.search(query, 'productos', max_results=5)

print(f"Resultados encontrados: {len(results)}\n")
if len(results) > 0:
    print(results[['id_producto', 'nombre', 'garantia_meses', 'precio_usd']].to_string(index=False))
else:
    print("No se encontraron resultados")

# ==================== FUNCIÓN HELPER ====================

def buscar_tabla(query: str, tabla: str = 'productos', max_results: int = 10):
    """
    Función helper para búsquedas rápidas en tablas

    Args:
        query: Consulta en lenguaje natural
        tabla: Nombre de la tabla (productos, ventas, etc)
        max_results: Número máximo de resultados
    """
    print(f"\n🔍 Buscando en tabla '{tabla}': '{query}'")
    print("-" * 60)

    results = table_searcher.search(query, tabla, max_results)

    print(f"\n📊 Encontrados {len(results)} resultados\n")

    if len(results) > 0:
        # Mostrar solo columnas relevantes
        if tabla == 'productos':
            cols = ['id_producto', 'nombre', 'marca', 'precio_usd', 'categoria']
        elif tabla == 'ventas':
            cols = ['id_venta', 'sucursal', 'total', 'metodo_pago', 'fecha']
        else:
            cols = results.columns.tolist()[:5]

        display_cols = [c for c in cols if c in results.columns]
        print(results[display_cols].to_string(index=False))

    return results

print("\n\n" + "="*60)
print("✅ PRUEBAS COMPLETADAS")
print("="*60)
print("\nFunción helper disponible:")
print("  buscar_tabla('tu consulta', tabla='productos', max_results=10)")
print("\nEjemplo:")
print("  results = buscar_tabla('licuadoras baratas', 'productos', 5)")


PRUEBAS DE LA BASE DE DATOS TABULAR

📝 PRUEBA 1: Filtro por precio
------------------------------------------------------------
Query: '¿Cuáles son las licuadoras de menos de $300?'

🔍 Filtros generados: {
  "filters": [
    {
      "column": "nombre",
      "operator": "contains",
      "value": "Licuadora"
    },
    {
      "column": "precio_usd",
      "operator": "<",
      "value": 300
    }
  ]
}
Resultados encontrados: 2

id_producto             nombre      marca  precio_usd categoria
      P0001          Licuadora   TechHome      283.63    Cocina
      P0004 Compacto Licuadora ChefMaster      259.42    Cocina


📝 PRUEBA 2: Filtro por marca
------------------------------------------------------------
Query: 'Productos de la marca TechHome'

🔍 Filtros generados: {
  "filters": [
    {
      "column": "marca",
      "operator": "==",
      "value": "TechHome"
    }
  ],
  "sort_by": "nombre",
  "ascending": true,
  "limit": 20
}
Resultados encontrados: 20

id_producto           

In [12]:
"""
============================================
CELDA 8: BASE DE DATOS DE GRAFOS (NEO4J)
============================================
Implementa base de datos de grafos con queries Cypher dinámicas usando GROQ
"""

from neo4j import GraphDatabase
from groq import Groq
import pandas as pd
import json
import re
from typing import List, Dict, Optional

print("\n" + "="*60)
print("CONFIGURANDO BASE DE DATOS DE GRAFOS")
print("="*60 + "\n")

# ==================== CONEXIÓN A NEO4J ====================

class Neo4jConnection:
    """Gestiona la conexión a Neo4j"""

    def __init__(self, uri: str, user: str, password: str):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        print(f"🔌 Conectando a Neo4j...")

        try:
            self.driver.verify_connectivity()
            print("✅ Conexión exitosa a Neo4j\n")
        except Exception as e:
            print(f"❌ Error conectando a Neo4j: {e}")
            raise

    def close(self):
        if self.driver:
            self.driver.close()

    def execute_query(self, query: str, parameters: Optional[Dict] = None) -> List[Dict]:
        """Ejecuta una query Cypher y retorna resultados"""
        with self.driver.session() as session:
            result = session.run(query, parameters or {})
            return [record.data() for record in result]

    def execute_write(self, query: str, parameters: Optional[Dict] = None):
        """Ejecuta una query de escritura"""
        with self.driver.session() as session:
            session.run(query, parameters or {})

    def clear_database(self):
        """Limpia toda la base de datos"""
        print("🗑️ Limpiando base de datos...")
        self.execute_write("MATCH (n) DETACH DELETE n")
        print("✅ Base de datos limpiada")

# ==================== CONSTRUCTOR DEL GRAFO ====================

class GraphBuilder:
    """Construye el grafo de conocimiento"""

    def __init__(self, neo4j_conn: Neo4jConnection):
        self.conn = neo4j_conn

    def build_graph(self, dataframes: Dict, documents: Dict):
        """Construye el grafo completo"""
        print("\n" + "="*60)
        print("CONSTRUYENDO GRAFO DE CONOCIMIENTO")
        print("="*60 + "\n")

        self.conn.clear_database()

        if 'productos' in dataframes:
            self._create_product_nodes(dataframes['productos'])
            self._create_category_and_brand_nodes(dataframes['productos'])

        if 'inventario' in dataframes:
            self._create_sucursal_nodes(dataframes['inventario'])
            self._create_inventory_relationships(dataframes['inventario'])

        if 'manuales' in documents:
            self._create_compatibility_relationships(documents['manuales'])

        self._create_indexes()

        print("\n✅ Grafo de conocimiento construido")
        self._print_statistics()

    def _create_product_nodes(self, df_productos: pd.DataFrame):
        """Crea nodos de productos"""
        print("📦 Creando nodos de productos...")

        query = """
        UNWIND $productos AS prod
        CREATE (p:Producto {
            id: prod.id_producto,
            nombre: prod.nombre,
            marca: prod.marca,
            precio: prod.precio_usd,
            stock: prod.stock,
            potencia: prod.potencia_w,
            voltaje: prod.voltaje,
            garantia_meses: prod.garantia_meses
        })
        """

        productos_data = df_productos.fillna('').to_dict('records')
        self.conn.execute_write(query, {'productos': productos_data})
        print(f"✅ {len(productos_data)} productos creados")

    def _create_category_and_brand_nodes(self, df_productos: pd.DataFrame):
        """Crea nodos de categorías y marcas + relaciones"""
        print("🏷️ Creando categorías, marcas y relaciones...")

        categorias = df_productos['categoria'].unique()
        for categoria in categorias:
            query = "MERGE (c:Categoria {nombre: $nombre})"
            self.conn.execute_write(query, {'nombre': categoria})

        marcas = df_productos['marca'].unique()
        for marca in marcas:
            query = "MERGE (m:Marca {nombre: $nombre})"
            self.conn.execute_write(query, {'nombre': marca})

        query = """
        MATCH (p:Producto), (c:Categoria), (m:Marca)
        WHERE p.id = $id_producto
          AND c.nombre = $categoria
          AND m.nombre = $marca
        MERGE (p)-[:PERTENECE_A]->(c)
        MERGE (p)-[:FABRICADO_POR]->(m)
        """

        for _, row in df_productos.iterrows():
            self.conn.execute_write(query, {
                'id_producto': row['id_producto'],
                'categoria': row['categoria'],
                'marca': row['marca']
            })

        print(f"✅ {len(categorias)} categorías, {len(marcas)} marcas")

    def _create_sucursal_nodes(self, df_inventario: pd.DataFrame):
        """Crea nodos de sucursales"""
        print("🏢 Creando nodos de sucursales...")

        sucursales = df_inventario['sucursal'].unique()
        for sucursal in sucursales:
            query = "MERGE (s:Sucursal {nombre: $nombre})"
            self.conn.execute_write(query, {'nombre': sucursal})

        print(f"✅ {len(sucursales)} sucursales creadas")

    def _create_inventory_relationships(self, df_inventario: pd.DataFrame):
        """Crea relaciones Producto -> Sucursal con stock"""
        print("📊 Creando relaciones de inventario...")

        query = """
        MATCH (p:Producto {id: $id_producto})
        MATCH (s:Sucursal {nombre: $sucursal})
        MERGE (p)-[r:DISPONIBLE_EN]->(s)
        SET r.stock = $stock,
            r.precio_sucursal = $precio_sucursal
        """

        count = 0
        for _, row in df_inventario.iterrows():
            self.conn.execute_write(query, {
                'id_producto': row['id_producto'],
                'sucursal': row['sucursal'],
                'stock': int(row['stock_sucursal']) if pd.notna(row['stock_sucursal']) else 0,
                'precio_sucursal': float(row['precio_sucursal']) if pd.notna(row['precio_sucursal']) else 0
            })
            count += 1

        print(f"✅ {count} relaciones de inventario creadas")

    def _create_compatibility_relationships(self, manuales: List[Dict]):
        """Extrae y crea relaciones de compatibilidad desde manuales"""
        print("🔗 Creando relaciones de compatibilidad...")

        count = 0
        for manual in manuales:
            if manual.get('tipo_seccion') != 'compatibilidad':
                continue

            id_producto_origen = manual['id_producto']
            contenido = manual['contenido']

            productos_encontrados = re.findall(r'P\d{4}', contenido)
            comparte_match = re.search(r'Comparte:\s*([^\n]+)', contenido)
            comparte = comparte_match.group(1).strip() if comparte_match else "Componente"

            for id_producto_destino in productos_encontrados:
                if id_producto_destino != id_producto_origen:
                    query = """
                    MATCH (p1:Producto {id: $id_origen})
                    MATCH (p2:Producto {id: $id_destino})
                    MERGE (p1)-[r:COMPATIBLE_CON]->(p2)
                    SET r.comparte = $comparte
                    """

                    try:
                        self.conn.execute_write(query, {
                            'id_origen': id_producto_origen,
                            'id_destino': id_producto_destino,
                            'comparte': comparte
                        })
                        count += 1
                    except:
                        pass

        print(f"✅ {count} relaciones de compatibilidad creadas")

    def _create_indexes(self):
        """Crea índices para mejor performance"""
        print("📇 Creando índices...")

        indexes = [
            "CREATE INDEX IF NOT EXISTS FOR (p:Producto) ON (p.id)",
            "CREATE INDEX IF NOT EXISTS FOR (c:Categoria) ON (c.nombre)",
            "CREATE INDEX IF NOT EXISTS FOR (m:Marca) ON (m.nombre)",
            "CREATE INDEX IF NOT EXISTS FOR (s:Sucursal) ON (s.nombre)"
        ]

        for index_query in indexes:
            try:
                self.conn.execute_write(index_query)
            except:
                pass

        print("✅ Índices creados")

    def _print_statistics(self):
        """Imprime estadísticas del grafo"""
        print("\n📊 Estadísticas del grafo:")

        stats = {
            'Productos': "MATCH (p:Producto) RETURN count(p) as count",
            'Categorías': "MATCH (c:Categoria) RETURN count(c) as count",
            'Marcas': "MATCH (m:Marca) RETURN count(m) as count",
            'Sucursales': "MATCH (s:Sucursal) RETURN count(s) as count",
            'Relaciones totales': "MATCH ()-[r]->() RETURN count(r) as count",
            'Compatibilidades': "MATCH ()-[r:COMPATIBLE_CON]->() RETURN count(r) as count"
        }

        for label, query in stats.items():
            result = self.conn.execute_query(query)
            count = result[0]['count'] if result else 0
            print(f"  - {label}: {count}")

# ==================== GENERADOR DE QUERIES CYPHER ====================

class CypherGenerator:
    """Genera queries Cypher a partir de lenguaje natural usando GROQ"""

    def __init__(self, api_key: str):
        self.client = Groq(api_key=api_key)
        print("🤖 Generador de Cypher inicializado")

    def generate_cypher(self, query: str) -> str:
        """Genera query Cypher desde lenguaje natural"""

        prompt = f"""Eres un experto en Neo4j y Cypher.

ESQUEMA DEL GRAFO:
- (Producto {{id, nombre, marca, precio, stock, potencia, voltaje, garantia_meses}})
- (Categoria {{nombre}})
- (Marca {{nombre}})
- (Sucursal {{nombre}})

RELACIONES:
- (Producto)-[:PERTENECE_A]->(Categoria)
- (Producto)-[:FABRICADO_POR]->(Marca)
- (Producto)-[:DISPONIBLE_EN {{stock, precio_sucursal}}]->(Sucursal)
- (Producto)-[:COMPATIBLE_CON {{comparte}}]->(Producto)

REGLAS DE SINTAXIS CYPHER:
1. Direcciones: (A)-[:REL]->(B) o (A)<-[:REL]-(B)
2. NUNCA uses: [:REL<-] o [:REL->] (sintaxis inválida)
3. Usa MATCH para buscar, CREATE para crear
4. Siempre limita resultados con LIMIT (máximo 20)
5. Usa DISTINCT para evitar duplicados

EJEMPLOS CORRECTOS:

Usuario: "¿Qué productos son compatibles con P0016?"
Cypher: MATCH (p1:Producto {{id: 'P0016'}})-[:COMPATIBLE_CON]->(p2:Producto) RETURN p2.nombre, p2.marca, p2.precio LIMIT 10

Usuario: "Productos de la marca TechHome"
Cypher: MATCH (p:Producto)-[:FABRICADO_POR]->(m:Marca {{nombre: 'TechHome'}}) RETURN p.nombre, p.precio, p.stock LIMIT 10

Usuario: "¿Dónde hay stock del producto P0001?"
Cypher: MATCH (p:Producto {{id: 'P0001'}})-[r:DISPONIBLE_EN]->(s:Sucursal) WHERE r.stock > 0 RETURN s.nombre, r.stock, r.precio_sucursal

Usuario: "Productos de la categoría Cocina"
Cypher: MATCH (p:Producto)-[:PERTENECE_A]->(c:Categoria {{nombre: 'Cocina'}}) RETURN p.nombre, p.marca, p.precio LIMIT 10

Usuario: "¿Qué marcas fabrican productos de Cocina?"
Cypher: MATCH (p:Producto)-[:PERTENECE_A]->(c:Categoria {{nombre: 'Cocina'}}), (p)-[:FABRICADO_POR]->(m:Marca) RETURN DISTINCT m.nombre LIMIT 10

Usuario: "Stock de TechHome por sucursal"
Cypher: MATCH (p:Producto)-[:FABRICADO_POR]->(m:Marca {{nombre: 'TechHome'}}), (p)-[r:DISPONIBLE_EN]->(s:Sucursal) WHERE r.stock > 0 RETURN s.nombre, p.nombre, r.stock ORDER BY s.nombre LIMIT 20

CONSULTA: "{query}"

Responde SOLO con la query Cypher válida, sin explicaciones ni markdown:"""

        try:
            response = self.client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=[
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1,
                max_tokens=300
            )

            cypher = response.choices[0].message.content.strip()

            # Limpiar markdown si existe
            cypher = re.sub(r'```cypher\s*', '', cypher)
            cypher = re.sub(r'```\s*', '', cypher)
            cypher = cypher.strip()

            return cypher
        except Exception as e:
            print(f"⚠️ Error generando Cypher: {e}")
            return ""

# ==================== MOTOR DE CONSULTAS ====================

class GraphSearcher:
    """Motor de consultas para el grafo"""

    def __init__(self, neo4j_conn: Neo4jConnection, cypher_gen: CypherGenerator):
        self.conn = neo4j_conn
        self.cypher_gen = cypher_gen
        print("🔍 Motor de búsqueda en grafos inicializado")

    def search(self, query: str) -> List[Dict]:
        """Consulta el grafo con lenguaje natural"""

        cypher = self.cypher_gen.generate_cypher(query)

        if not cypher:
            return []

        print(f"🔍 Cypher generado:\n{cypher}\n")

        try:
            results = self.conn.execute_query(cypher)
            return results
        except Exception as e:
            print(f"❌ Error ejecutando Cypher: {e}")
            return []

    def format_results(self, results: List[Dict]) -> str:
        """Formatea resultados para mostrar"""
        if not results:
            return "No se encontraron resultados."

        output = f"Se encontraron {len(results)} resultados:\n\n"

        for i, result in enumerate(results[:10], 1):
            output += f"{i}. "
            output += " | ".join([f"{k}: {v}" for k, v in result.items()])
            output += "\n"

        if len(results) > 10:
            output += f"\n... y {len(results) - 10} más"

        return output

# ==================== INICIALIZACIÓN ====================

print("🔧 Inicializando base de datos de grafos...\n")

# Conectar a Neo4j
neo4j_conn = Neo4jConnection(
    uri=config.NEO4J_URI,
    user=config.NEO4J_USER,
    password=config.NEO4J_PASSWORD
)

# Construir grafo
graph_builder = GraphBuilder(neo4j_conn)
graph_builder.build_graph(dataframes, documents)

# Crear generador de Cypher y motor de consultas
cypher_generator = CypherGenerator(config.GROQ_API_KEY)
graph_searcher = GraphSearcher(neo4j_conn, cypher_generator)

print("\n" + "="*60)
print("✅ BASE DE DATOS DE GRAFOS LISTA")
print("="*60)
print("""
Interfaz de búsqueda:
    results = graph_searcher.search("¿Qué productos son compatibles con P0016?")
""")
print("="*60 + "\n")


CONFIGURANDO BASE DE DATOS DE GRAFOS

🔧 Inicializando base de datos de grafos...

🔌 Conectando a Neo4j...
✅ Conexión exitosa a Neo4j


CONSTRUYENDO GRAFO DE CONOCIMIENTO

🗑️ Limpiando base de datos...
✅ Base de datos limpiada
📦 Creando nodos de productos...
✅ 300 productos creados
🏷️ Creando categorías, marcas y relaciones...
✅ 4 categorías, 17 marcas
🏢 Creando nodos de sucursales...
✅ 24 sucursales creadas
📊 Creando relaciones de inventario...
✅ 4100 relaciones de inventario creadas
🔗 Creando relaciones de compatibilidad...
✅ 250 relaciones de compatibilidad creadas
📇 Creando índices...
✅ Índices creados

✅ Grafo de conocimiento construido

📊 Estadísticas del grafo:
  - Productos: 300
  - Categorías: 4
  - Marcas: 17
  - Sucursales: 24
  - Relaciones totales: 4950
  - Compatibilidades: 250
🤖 Generador de Cypher inicializado
🔍 Motor de búsqueda en grafos inicializado

✅ BASE DE DATOS DE GRAFOS LISTA

Interfaz de búsqueda:
    results = graph_searcher.search("¿Qué productos son compati

In [13]:
"""
============================================
CELDA 9: PRUEBAS DE BASE DE GRAFOS
============================================
"""

print("\n" + "="*60)
print("PRUEBAS DE LA BASE DE DATOS DE GRAFOS")
print("="*60 + "\n")

# ==================== PRUEBA 1: COMPATIBILIDAD ====================

print("📝 PRUEBA 1: Productos compatibles")
print("-" * 60)

query = "¿Qué productos son compatibles con P0016?"
print(f"Query: '{query}'\n")

results = graph_searcher.search(query)
formatted = graph_searcher.format_results(results)
print(formatted)

# ==================== PRUEBA 2: FILTRO POR MARCA EN GRAFO ====================

print("\n\n" + "="*60)
print("📝 PRUEBA 2: Productos por marca")
print("-" * 60)

query = "Productos de la marca TechHome"
print(f"Query: '{query}'\n")

results = graph_searcher.search(query)
formatted = graph_searcher.format_results(results)
print(formatted)

# ==================== PRUEBA 3: STOCK POR SUCURSAL ====================

print("\n\n" + "="*60)
print("📝 PRUEBA 3: Stock por sucursal")
print("-" * 60)

query = "¿Dónde hay stock del producto P0001?"
print(f"Query: '{query}'\n")

results = graph_searcher.search(query)
formatted = graph_searcher.format_results(results)
print(formatted)

# ==================== PRUEBA 4: PRODUCTOS POR CATEGORÍA ====================

print("\n\n" + "="*60)
print("📝 PRUEBA 4: Productos por categoría")
print("-" * 60)

query = "¿Qué productos pertenecen a la categoría Cocina?"
print(f"Query: '{query}'\n")

results = graph_searcher.search(query)
formatted = graph_searcher.format_results(results)
print(formatted)

# ==================== FUNCIÓN HELPER ====================

def buscar_grafo(query: str):
    """
    Función helper para búsquedas rápidas en el grafo

    Args:
        query: Consulta en lenguaje natural
    """
    print(f"\n🔍 Buscando en grafo: '{query}'")
    print("-" * 60)

    results = graph_searcher.search(query)
    formatted = graph_searcher.format_results(results)
    print(formatted)

    return results

print("\n\n" + "="*60)
print("✅ PRUEBAS COMPLETADAS")
print("="*60)
print("\nFunción helper disponible:")
print("  buscar_grafo('tu consulta')")
print("\nEjemplo:")
print("  results = buscar_grafo('productos compatibles con P0020')")


PRUEBAS DE LA BASE DE DATOS DE GRAFOS

📝 PRUEBA 1: Productos compatibles
------------------------------------------------------------
Query: '¿Qué productos son compatibles con P0016?'

🔍 Cypher generado:
MATCH (p1:Producto {id: 'P0016'})-[:COMPATIBLE_CON]->(p2:Producto) RETURN DISTINCT p2.nombre, p2.marca, p2.precio LIMIT 10

Se encontraron 5 resultados:

1. p2.nombre: Eco Mixer II | p2.marca: CookElite | p2.precio: 2906.9
2. p2.nombre: Deluxe Abridor de Latas | p2.marca: CookElite | p2.precio: 688.19
3. p2.nombre: Pava Eléctrica | p2.marca: CookElite | p2.precio: 1051.88
4. p2.nombre: Sandwichera | p2.marca: CookElite | p2.precio: 565.96
5. p2.nombre: Olla de Cocción Lenta II | p2.marca: CookElite | p2.precio: 1667.94



📝 PRUEBA 2: Productos por marca
------------------------------------------------------------
Query: 'Productos de la marca TechHome'

🔍 Cypher generado:
MATCH (p:Producto)-[:FABRICADO_POR]->(m:Marca {nombre: 'TechHome'}) RETURN p.nombre, p.precio, p.stock LIMIT 10



In [14]:
"""
============================================
CELDA 10: CLASIFICADOR DE INTENCIÓN
============================================
Compara dos enfoques:
1. Clasificador basado en Keywords (baseline)
2. Clasificador basado en LLM Few-Shot (GROQ)
"""

import numpy as np
from typing import List
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from groq import Groq

print("\n" + "="*60)
print("CLASIFICADOR DE INTENCIÓN")
print("="*60 + "\n")

# ==================== DATASET SINTÉTICO ====================

def create_synthetic_dataset() -> tuple:
    """Crea dataset sintético para entrenamiento y evaluación"""

    # Ejemplos de cada clase
    vectorial_queries = [
        "¿Cómo uso mi licuadora para hacer smoothies?",
        "¿Qué opinan los usuarios de esta cafetera?",
        "Mi licuadora no enciende, ¿qué hago?",
        "¿Cómo limpio mi procesadora?",
        "¿Es normal que mi batidora haga ruido?",
        "Instrucciones para usar la picadora",
        "¿Qué dicen las reseñas del producto P0001?",
        "¿Cómo se mantiene la licuadora?",
        "Problemas comunes con el exprimidor",
        "Manual de uso de la batidora",
        "¿Cómo preparar masa con la procesadora?",
        "Opiniones sobre las licuadoras TechHome",
        "¿Qué hacer si mi producto tiene fugas?",
        "¿Cómo funciona el modo pulse?",
        "Feedback de usuarios sobre cafeteras",
        "¿Es seguro usar la picadora continuamente?",
        "¿Cómo picar vegetales correctamente?",
        "Reseñas de la marca ChefMaster",
        "¿Por qué mi licuadora vibra mucho?",
        "¿Cómo hacer smoothies cremosos?"
    ]

    tabular_queries = [
        "¿Cuáles son las licuadoras de menos de $200?",
        "Mostrar productos de la marca TechHome",
        "¿Qué productos tienen garantía mayor a 24 meses?",
        "Licuadoras con voltaje 220V",
        "Productos en stock",
        "¿Cuánto cuesta la procesadora P0013?",
        "Productos con potencia mayor a 1000W",
        "Filtrar por color rojo",
        "¿Qué hay disponible en menos de $150?",
        "Productos con capacidad de 2 litros",
        "Mostrar batidoras baratas",
        "¿Cuál es el precio de la licuadora compacta?",
        "Productos de la categoría Cocina",
        "Filtrar por marca HomeChef",
        "¿Qué productos cuestan menos de $100?",
        "Stock de productos en sucursal Centro",
        "Productos con 36 meses de garantía",
        "¿Cuántas ventas hubo en noviembre?",
        "Distribución de ventas por método de pago",
        "Top 10 productos más vendidos"
    ]

    grafo_queries = [
        "¿Qué productos son compatibles con P0016?",
        "Productos relacionados con licuadoras",
        "¿Qué accesorios hay para la batidora?",
        "Productos similares al P0001",
        "¿Qué repuestos comparte con otros productos?",
        "¿Dónde hay stock del producto P0005?",
        "Productos de la misma categoría que P0013",
        "¿Qué productos usan el mismo motor?",
        "Accesorios compatibles con la picadora",
        "¿En qué sucursales está disponible P0020?",
        "Productos relacionados en la categoría cocina",
        "¿Qué productos comparten componentes con P0008?",
        "¿Qué marca fabrica productos similares?",
        "Productos compatibles de TechHome",
        "¿Dónde puedo conseguir repuestos?",
        "Productos que comparten cuchillas",
        "¿Qué otros productos de CookElite son compatibles?",
        "Stock por sucursal del producto P0001",
        "Productos relacionados con procesadoras",
        "¿Qué accesorios son intercambiables?"
    ]

    # Crear dataset
    queries = vectorial_queries + tabular_queries + grafo_queries
    labels = (
        ['vectorial'] * len(vectorial_queries) +
        ['tabular'] * len(tabular_queries) +
        ['grafo'] * len(grafo_queries)
    )

    return queries, labels

# ==================== CLASIFICADOR 1: KEYWORDS (BASELINE) ====================

class KeywordClassifier:
    """Clasificador simple basado en palabras clave"""

    def __init__(self):
        self.vectorial_keywords = [
            'cómo', 'como', 'usar', 'funciona', 'problema', 'opinión',
            'reseña', 'manual', 'limpiar', 'mantener', 'instrucciones',
            'normal', 'seguro', 'hacer', 'preparar', 'feedback', 'dicen',
            'opina', 'opinan', 'comentario', 'review'
        ]
        self.tabular_keywords = [
            'precio', 'menos', 'mayor', 'stock', 'cuánto', 'filtrar',
            'ventas', 'garantía', 'cuesta', 'disponible', 'voltaje',
            'potencia', 'capacidad', 'color', 'categoría', 'marca',
            'baratas', 'barato', 'top', 'distribución', 'cantidad',
            'cuántos', 'cuántas', 'hay', 'listar', 'mostrar', 'todos',
            'por sucursal', 'por marca', 'por categoría', 'agrupado'
        ]
        self.grafo_keywords = [
            'compatible', 'relacionado', 'similar', 'accesorio', 'repuesto',
            'donde', 'dónde', 'sucursal', 'comparten', 'mismo', 'misma',
            'intercambiable', 'componente', 'motor', 'cuchilla', 'específico',
            'producto P', 'P0', 'con el producto'
        ]

    def predict(self, query: str) -> str:
        """Predice la clase de una consulta"""
        query_lower = query.lower()

        vectorial_score = sum(1 for kw in self.vectorial_keywords if kw in query_lower)
        tabular_score = sum(1 for kw in self.tabular_keywords if kw in query_lower)
        grafo_score = sum(1 for kw in self.grafo_keywords if kw in query_lower)

        scores = {
            'vectorial': vectorial_score,
            'tabular': tabular_score,
            'grafo': grafo_score
        }

        return max(scores, key=scores.get)

    def predict_batch(self, queries: List[str]) -> List[str]:
        """Predice múltiples consultas"""
        return [self.predict(q) for q in queries]

# ==================== CLASIFICADOR 2: LLM FEW-SHOT ====================

class LLMClassifier:
    """Clasificador usando GROQ con Few-Shot Learning"""

    def __init__(self, api_key: str):
        self.client = Groq(api_key=api_key)

        self.prompt_template = """Clasifica la siguiente consulta en UNA de estas categorías:

CATEGORÍAS:
- vectorial: Preguntas sobre USO, FUNCIONAMIENTO, PROBLEMAS, MANTENIMIENTO, OPINIONES de productos
- tabular: Consultas de PRECIOS, FILTROS por características, STOCK, ESPECIFICACIONES, VENTAS, AGRUPACIONES, LISTADOS
- grafo: Productos RELACIONADOS, COMPATIBILIDAD, ACCESORIOS, SIMILARES con PRODUCTOS ESPECÍFICOS (IDs como P0001)

REGLAS:
- Si pregunta por "marcas disponibles", "productos por sucursal", "cuántos/cuántas" → tabular
- Si menciona ID específico (P0001, P0016) con "compatible" o "relacionado" → grafo
- Si pregunta "cómo usar", "opiniones", "problemas" → vectorial

EJEMPLOS:

Consulta: "¿Cómo uso mi licuadora para hacer smoothies?"
Categoría: vectorial

Consulta: "¿Cuáles son las licuadoras de menos de $200?"
Categoría: tabular

Consulta: "¿Qué productos son compatibles con P0016?"
Categoría: grafo

Consulta: "¿Qué opinan los usuarios de esta cafetera?"
Categoría: vectorial

Consulta: "Productos con voltaje 220V"
Categoría: tabular

Consulta: "¿Dónde hay stock del producto P0001?"
Categoría: grafo

Consulta: "¿Qué marcas de heladeras hay disponibles por sucursal?"
Categoría: tabular

Consulta: "Productos relacionados con licuadoras"
Categoría: tabular

Consulta: "¿Qué repuestos comparte el producto P0005 con otros?"
Categoría: grafo

CONSULTA: "{query}"

Responde SOLO con una palabra (vectorial, tabular o grafo):"""

    def predict(self, query: str) -> str:
        """Predice la clase de una consulta"""
        prompt = self.prompt_template.format(query=query)

        try:
            response = self.client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=[
                    {"role": "user", "content": prompt}
                ],
                temperature=0.1,
                max_tokens=10
            )

            prediction = response.choices[0].message.content.strip().lower()

            # Validar que sea una clase válida
            if prediction in ['vectorial', 'tabular', 'grafo']:
                return prediction
            else:
                # Fallback a keywords
                return self._keyword_fallback(query)

        except Exception as e:
            print(f"⚠️ Error en LLM: {e}")
            return self._keyword_fallback(query)

    def _keyword_fallback(self, query: str) -> str:
        """Clasificación por palabras clave como fallback"""
        query_lower = query.lower()

        vectorial_keywords = ['cómo', 'como', 'usar', 'funciona', 'problema', 'opinión', 'reseña']
        tabular_keywords = ['precio', 'menos', 'mayor', 'stock', 'cuánto', 'filtrar']
        grafo_keywords = ['compatible', 'relacionado', 'similar', 'accesorio', 'donde', 'sucursal']

        vectorial_score = sum(1 for kw in vectorial_keywords if kw in query_lower)
        tabular_score = sum(1 for kw in tabular_keywords if kw in query_lower)
        grafo_score = sum(1 for kw in grafo_keywords if kw in query_lower)

        scores = {
            'vectorial': vectorial_score,
            'tabular': tabular_score,
            'grafo': grafo_score
        }

        return max(scores, key=scores.get)

    def predict_batch(self, queries: List[str]) -> List[str]:
        """Predice múltiples consultas"""
        return [self.predict(q) for q in queries]

# ==================== EVALUACIÓN ====================

def evaluate_classifier(classifier, X_test: List[str], y_test: List[str], name: str):
    """Evalúa un clasificador y muestra métricas"""

    print(f"\n{'='*60}")
    print(f"EVALUACIÓN: {name}")
    print(f"{'='*60}\n")

    # Predicciones
    y_pred = classifier.predict_batch(X_test)

    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test, y_pred,
        average='weighted',
        zero_division=0
    )

    print(f"📊 Métricas globales:")
    print(f"  Accuracy:  {accuracy:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    print(f"  F1-Score:  {f1:.3f}")

    print(f"\n📋 Reporte por clase:")
    print(classification_report(y_test, y_pred, zero_division=0))

    # Ejemplos de predicciones
    print(f"🔍 Ejemplos de predicciones:")
    for i in range(min(5, len(X_test))):
        emoji = "✅" if y_pred[i] == y_test[i] else "❌"
        print(f"  {emoji} '{X_test[i][:50]}...'")
        print(f"     Esperado: {y_test[i]} | Predicho: {y_pred[i]}")

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'predictions': y_pred
    }

# ==================== EJECUCIÓN ====================

print("📊 Creando dataset sintético...")
queries, labels = create_synthetic_dataset()
print(f"✅ Dataset creado: {len(queries)} consultas")
print(f"  - Vectorial: {labels.count('vectorial')}")
print(f"  - Tabular: {labels.count('tabular')}")
print(f"  - Grafo: {labels.count('grafo')}")

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    queries, labels,
    test_size=0.3,
    random_state=42,
    stratify=labels
)

print(f"\n✅ Split completado:")
print(f"  Train: {len(X_train)} consultas")
print(f"  Test: {len(X_test)} consultas")

# Clasificador 1: Keywords
print("\n🔧 Evaluando Clasificador Baseline (Keywords)...")
keyword_classifier = KeywordClassifier()
results_baseline = evaluate_classifier(keyword_classifier, X_test, y_test, "Baseline (Keywords)")

# Clasificador 2: LLM Few-Shot
print("\n🔧 Evaluando Clasificador LLM (Few-Shot)...")
llm_classifier = LLMClassifier(config.GROQ_API_KEY)
results_llm = evaluate_classifier(llm_classifier, X_test, y_test, "LLM Few-Shot (GROQ)")

# Comparación final
print("\n" + "="*60)
print("📊 COMPARACIÓN FINAL")
print("="*60 + "\n")

import pandas as pd
comparison = pd.DataFrame({
    'Clasificador': ['Baseline (Keywords)', 'LLM Few-Shot'],
    'Accuracy': [results_baseline['accuracy'], results_llm['accuracy']],
    'Precision': [results_baseline['precision'], results_llm['precision']],
    'Recall': [results_baseline['recall'], results_llm['recall']],
    'F1-Score': [results_baseline['f1'], results_llm['f1']]
})

print(comparison.to_string(index=False))

# Seleccionar mejor clasificador
best_classifier_name = 'LLM Few-Shot' if results_llm['f1'] > results_baseline['f1'] else 'Baseline'
best_classifier = llm_classifier if results_llm['f1'] > results_baseline['f1'] else keyword_classifier

print(f"\n🏆 MEJOR CLASIFICADOR: {best_classifier_name}")
print(f"   F1-Score: {max(results_llm['f1'], results_baseline['f1']):.3f}")

# Guardar el mejor clasificador
intent_classifier = best_classifier

print("\n" + "="*60)
print("✅ CLASIFICADOR DE INTENCIÓN LISTO")
print("="*60)
print(f"\nUso: intent_classifier.predict('tu consulta')")
print(f"Clasificador seleccionado: {best_classifier_name}")
print("="*60 + "\n")


CLASIFICADOR DE INTENCIÓN

📊 Creando dataset sintético...
✅ Dataset creado: 60 consultas
  - Vectorial: 20
  - Tabular: 20
  - Grafo: 20

✅ Split completado:
  Train: 42 consultas
  Test: 18 consultas

🔧 Evaluando Clasificador Baseline (Keywords)...

EVALUACIÓN: Baseline (Keywords)

📊 Métricas globales:
  Accuracy:  0.833
  Precision: 0.889
  Recall:    0.833
  F1-Score:  0.822

📋 Reporte por clase:
              precision    recall  f1-score   support

       grafo       1.00      0.50      0.67         6
     tabular       0.67      1.00      0.80         6
   vectorial       1.00      1.00      1.00         6

    accuracy                           0.83        18
   macro avg       0.89      0.83      0.82        18
weighted avg       0.89      0.83      0.82        18

🔍 Ejemplos de predicciones:
  ✅ '¿Cómo limpio mi procesadora?...'
     Esperado: vectorial | Predicho: vectorial
  ✅ 'Mostrar productos de la marca TechHome...'
     Esperado: tabular | Predicho: tabular
  ❌ 'Produc

In [ ]:
"""
============================================
CELDA 11: PIPELINE DE RECUPERACIÓN
============================================
Integra las 3 fuentes de datos:
- Vectorial (ChromaDB + búsqueda híbrida)
- Tabular (Pandas con filtros dinámicos)
- Grafo (Neo4j con Cypher dinámico)
"""

from typing import Dict, List, Optional
import json

print("\n" + "="*60)
print("PIPELINE DE RECUPERACIÓN INTEGRADO")
print("="*60 + "\n")

# ==================== PIPELINE DE RECUPERACIÓN ====================

class RetrievalPipeline:
    """
    Pipeline que integra las 3 fuentes de datos y el clasificador de intención
    """

    def __init__(
        self,
        vector_search,
        table_search,
        graph_search,
        classifier
    ):
        self.vector_search = vector_search
        self.table_search = table_search
        self.graph_search = graph_search
        self.classifier = classifier

        print("🔧 Pipeline de recuperación inicializado")
        print("   ✅ Búsqueda vectorial (ChromaDB)")
        print("   ✅ Búsqueda tabular (Pandas)")
        print("   ✅ Búsqueda en grafos (Neo4j)")
        print("   ✅ Clasificador de intención")

    def retrieve(self, query: str, top_k: int = 5) -> Dict:
        """
        Recupera información relevante según la consulta

        Args:
            query: Consulta del usuario
            top_k: Número de resultados a retornar

        Returns:
            Dict con: intent, source, results, context
        """

        # 1. Clasificar intención
        intent = self.classifier.predict(query)
        print(f"\n🎯 Intención detectada: {intent}")

        # 2. Recuperar según intención
        if intent == 'vectorial':
            results = self._retrieve_vectorial(query, top_k)
            source = 'vectorial'
        elif intent == 'tabular':
            results = self._retrieve_tabular(query, top_k)
            source = 'tabular'
        elif intent == 'grafo':
            results = self._retrieve_grafo(query, top_k)
            source = 'grafo'
        else:
            results = []
            source = 'unknown'

        # 3. Formatear contexto
        context = self._format_context(results, source)

        # 4. Calcular número de resultados
        import pandas as pd
        if isinstance(results, pd.DataFrame):
            num_results = len(results)
        elif isinstance(results, list):
            num_results = len(results)
        else:
            num_results = 0

        return {
            'intent': intent,
            'source': source,
            'results': results,
            'context': context,
            'num_results': num_results
        }

    def _retrieve_vectorial(self, query: str, top_k: int) -> List[Dict]:
        """Recupera de la base vectorial"""
        print(f"📚 Buscando en base vectorial (híbrida + rerank)...")

        results = self.vector_search.search(query, top_k=top_k)

        print(f"   ✅ {len(results)} documentos recuperados")

        return results

    def _retrieve_tabular(self, query: str, top_k: int):
        """Recupera de las tablas"""
        print(f"📊 Buscando en base tabular (Pandas)...")

        # Intentar búsqueda en productos primero
        results = self.table_search.search(query, 'productos', max_results=top_k)

        # Si no hay resultados, intentar en ventas
        if len(results) == 0:
            results = self.table_search.search(query, 'ventas', max_results=top_k)

        print(f"   ✅ {len(results)} filas recuperadas")

        return results

    def _retrieve_grafo(self, query: str, top_k: int = 10) -> List[Dict]:
        """Recupera del grafo"""
        print(f"🕸️ Buscando en base de grafos (Neo4j)...")

        results = self.graph_search.search(query)

        # Limitar resultados
        if len(results) > top_k:
            results = results[:top_k]

        print(f"   ✅ {len(results)} nodos/relaciones recuperados")

        return results

    def _format_context(self, results, source: str) -> str:
        """Formatea los resultados como contexto para el LLM"""

        import pandas as pd

        # Verificar si hay resultados según el tipo
        is_empty = False
        if isinstance(results, pd.DataFrame):
            is_empty = results.empty
        elif isinstance(results, list):
            is_empty = len(results) == 0
        else:
            is_empty = not results

        if is_empty:
            return "No se encontró información relevante para responder esta consulta."

        context = f"INFORMACIÓN RECUPERADA (Fuente: {source}):\n\n"

        if source == 'vectorial':
            # Resultados de ChromaDB
            for i, result in enumerate(results, 1):
                context += f"{i}. {result['text'][:300]}...\n"
                context += f"   [Fuente: {result['metadata'].get('source', 'N/A')}]\n\n"

        elif source == 'tabular':
            # Resultados de Pandas DataFrame
            if isinstance(results, pd.DataFrame):
                # Convertir a diccionarios
                records = results.head(10).to_dict('records')
                for i, record in enumerate(records, 1):
                    context += f"{i}. "
                    context += " | ".join([f"{k}: {v}" for k, v in record.items() if pd.notna(v)])
                    context += "\n"
            else:
                context += str(results)

        elif source == 'grafo':
            # Resultados de Neo4j
            for i, result in enumerate(results, 1):
                context += f"{i}. "
                context += " | ".join([f"{k}: {v}" for k, v in result.items()])
                context += "\n"

        return context.strip()

    def get_stats(self) -> Dict:
        """Retorna estadísticas del pipeline"""
        return {
            'classifier': type(self.classifier).__name__,
            'vector_db': 'ChromaDB + Hybrid Search',
            'table_db': 'Pandas DataFrames',
            'graph_db': 'Neo4j'
        }

# ==================== INICIALIZACIÓN ====================

print("🔧 Inicializando pipeline de recuperación...\n")

# Crear pipeline con todos los componentes
retrieval_pipeline = RetrievalPipeline(
    vector_search=hybrid_search,      # De celda 4
    table_search=table_searcher,      # De celda 6
    graph_search=graph_searcher,      # De celda 8
    classifier=intent_classifier      # De celda 10
)

print("\n" + "="*60)
print("✅ PIPELINE DE RECUPERACIÓN LISTO")
print("="*60)

# Mostrar estadísticas
stats = retrieval_pipeline.get_stats()
print(f"""
Componentes integrados:
  - Clasificador: {stats['classifier']}
  - Base vectorial: {stats['vector_db']}
  - Base tabular: {stats['table_db']}
  - Base de grafos: {stats['graph_db']}

Uso:
  result = retrieval_pipeline.retrieve("¿Cómo usar mi licuadora?", top_k=5)

  print(result['intent'])    # Intención detectada
  print(result['source'])    # Fuente de datos usada
  print(result['context'])   # Contexto formateado para LLM
""")

print("="*60 + "\n")


PIPELINE DE RECUPERACIÓN INTEGRADO

🔧 Inicializando pipeline de recuperación...

🔧 Pipeline de recuperación inicializado
   ✅ Búsqueda vectorial (ChromaDB)
   ✅ Búsqueda tabular (Pandas)
   ✅ Búsqueda en grafos (Neo4j)
   ✅ Clasificador de intención

✅ PIPELINE DE RECUPERACIÓN LISTO

Componentes integrados:
  - Clasificador: KeywordClassifier
  - Base vectorial: ChromaDB + Hybrid Search
  - Base tabular: Pandas DataFrames
  - Base de grafos: Neo4j

Uso:
  result = retrieval_pipeline.retrieve("¿Cómo usar mi licuadora?", top_k=5)

  print(result['intent'])    # Intención detectada
  print(result['source'])    # Fuente de datos usada
  print(result['context'])   # Contexto formateado para LLM




In [ ]:
"""
============================================
CELDA 12: PRUEBAS DEL PIPELINE DE RECUPERACIÓN
============================================
"""

print("\n" + "="*60)
print("PRUEBAS DEL PIPELINE DE RECUPERACIÓN")
print("="*60 + "\n")

# ==================== PRUEBA 1: CONSULTA VECTORIAL ====================

print("📝 PRUEBA 1: Consulta vectorial (uso de producto)")
print("-" * 60)

query = "¿Cómo uso mi licuadora para hacer smoothies?"
print(f"Query: '{query}'\n")

result = retrieval_pipeline.retrieve(query, top_k=3)

print(f"\n📋 Resultado:")
print(f"  Intención: {result['intent']}")
print(f"  Fuente: {result['source']}")
print(f"  Documentos recuperados: {result['num_results']}")
print(f"\n📄 Contexto generado:")
print(result['context'][:500] + "..." if len(result['context']) > 500 else result['context'])

# ==================== PRUEBA 2: CONSULTA TABULAR ====================

print("\n\n" + "="*60)
print("📝 PRUEBA 2: Consulta tabular (filtro de precios)")
print("-" * 60)

query = "¿Cuáles son las licuadoras de menos de $300?"
print(f"Query: '{query}'\n")

result = retrieval_pipeline.retrieve(query, top_k=5)

print(f"\n📋 Resultado:")
print(f"  Intención: {result['intent']}")
print(f"  Fuente: {result['source']}")
print(f"  Productos recuperados: {result['num_results']}")
print(f"\n📄 Contexto generado:")
print(result['context'][:500] + "..." if len(result['context']) > 500 else result['context'])

# ==================== PRUEBA 3: CONSULTA GRAFO ====================

print("\n\n" + "="*60)
print("📝 PRUEBA 3: Consulta grafo (compatibilidad)")
print("-" * 60)

query = "¿Qué productos son compatibles con P0016?"
print(f"Query: '{query}'\n")

result = retrieval_pipeline.retrieve(query, top_k=5)

print(f"\n📋 Resultado:")
print(f"  Intención: {result['intent']}")
print(f"  Fuente: {result['source']}")
print(f"  Relaciones recuperadas: {result['num_results']}")
print(f"\n📄 Contexto generado:")
print(result['context'][:500] + "..." if len(result['context']) > 500 else result['context'])

# ==================== PRUEBA 4: OPINIONES ====================

print("\n\n" + "="*60)
print("📝 PRUEBA 4: Consulta vectorial (opiniones)")
print("-" * 60)

query = "¿Qué opinan los usuarios de las licuadoras TechHome?"
print(f"Query: '{query}'\n")

result = retrieval_pipeline.retrieve(query, top_k=3)

print(f"\n📋 Resultado:")
print(f"  Intención: {result['intent']}")
print(f"  Fuente: {result['source']}")
print(f"  Documentos recuperados: {result['num_results']}")
print(f"\n📄 Contexto generado:")
print(result['context'][:500] + "..." if len(result['context']) > 500 else result['context'])

# ==================== FUNCIÓN HELPER ====================

def buscar(query: str, top_k: int = 5):
    """
    Función helper para búsquedas rápidas

    Args:
        query: Consulta del usuario
        top_k: Número de resultados
    """
    print(f"\n🔍 Buscando: '{query}'")
    print("-" * 60)

    result = retrieval_pipeline.retrieve(query, top_k)

    print(f"\n📊 Resumen:")
    print(f"  Intención: {result['intent']}")
    print(f"  Fuente: {result['source']}")
    print(f"  Resultados: {result['num_results']}")
    print(f"\n💬 Contexto:")
    print(result['context'][:300] + "..." if len(result['context']) > 300 else result['context'])

    return result

print("\n\n" + "="*60)
print("✅ PRUEBAS COMPLETADAS")
print("="*60)
print("\nFunción helper disponible:")
print("  buscar('tu consulta', top_k=5)")
print("\nEjemplo:")
print("  result = buscar('productos con garantía mayor a 24 meses')")


PRUEBAS DEL PIPELINE DE RECUPERACIÓN

📝 PRUEBA 1: Consulta vectorial (uso de producto)
------------------------------------------------------------
Query: '¿Cómo uso mi licuadora para hacer smoothies?'


🎯 Intención detectada: vectorial
📚 Buscando en base vectorial (híbrida + rerank)...
   ✅ 3 documentos recuperados

📋 Resultado:
  Intención: vectorial
  Fuente: vectorial
  Documentos recuperados: 3

📄 Contexto generado:
INFORMACIÓN RECUPERADA (Fuente: vectorial):

1. INSTRUCCIONES DE USO - Pregunta: ¿Cómo se usa correctamente este producto?
Respuesta: El Licuadora de TechHome está diseñado para uso doméstico. Revise el manual del producto (código P0001) para más detalles. Ante cualquier duda, contacte a nuestro servicio de atención al cliente....
   [Fuente: faq]

2. INSTRUCCIONES DE USO - Pregunta: ¿Cómo se usa correctamente este producto?
Respuesta: El Licuadora de TechHome está diseñado para uso doméstico. R...


📝 PRUEBA 2: Consulta tabular (filtro de precios)
-------------------

In [ ]:
"""
============================================
CELDA 13: LLM GENERATOR
============================================
Wrapper unificado para generación de respuestas con GROQ
Soporta múltiples modelos y manejo de errores
"""

from groq import Groq
from typing import List, Dict, Optional
import time

print("\n" + "="*60)
print("LLM GENERATOR")
print("="*60 + "\n")

# ==================== LLM GENERATOR ====================

class LLMGenerator:
    """
    Generador de respuestas usando GROQ (LLama 3.3 70B)
    """

    def __init__(
        self,
        api_key: str,
        model: str = "llama-3.3-70b-versatile",
        temperature: float = 0.7,
        max_tokens: int = 1024
    ):
        self.client = Groq(api_key=api_key)
        self.model = model
        self.temperature = temperature
        self.max_tokens = max_tokens

        print(f"🤖 LLM Generator inicializado")
        print(f"   Proveedor: GROQ")
        print(f"   Modelo: {model}")
        print(f"   Temperature: {temperature}")
        print(f"   Max tokens: {max_tokens}")

    def generate(
        self,
        query: str,
        context: str,
        system_prompt: Optional[str] = None,
        conversational_history: Optional[List[Dict]] = None
    ) -> Dict:
        """
        Genera una respuesta basada en el contexto

        Args:
            query: Pregunta del usuario
            context: Contexto recuperado
            system_prompt: Prompt del sistema (opcional)
            conversational_history: Historial de conversación (opcional)

        Returns:
            Dict con: response, model, tokens, time
        """

        # System prompt por defecto
        if system_prompt is None:
            system_prompt = """Eres un asistente virtual experto en electrodomésticos.

Instrucciones:
1. Responde SIEMPRE en español
2. Usa SOLO la información del contexto proporcionado
3. Si no hay información suficiente, dilo claramente y sugiere reformular la pregunta
4. Sé claro, conciso y útil
5. Cita información específica cuando sea relevante (ej: "El producto P0001...")
6. Mantén un tono profesional pero amigable

IMPORTANTE: NO inventes información. Si el contexto no contiene la respuesta, indícalo."""

        # Construir mensajes
        messages = [
            {"role": "system", "content": system_prompt}
        ]

        # Agregar historial conversacional si existe
        if conversational_history:
            messages.extend(conversational_history)

        # Agregar query actual con contexto
        user_message = f"""CONTEXTO:
{context}

PREGUNTA DEL USUARIO:
{query}

Responde basándote SOLO en el contexto proporcionado:"""

        messages.append({"role": "user", "content": user_message})

        # Generar respuesta
        start_time = time.time()

        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=self.temperature,
                max_tokens=self.max_tokens,
                top_p=0.9
            )

            elapsed_time = time.time() - start_time

            return {
                'response': response.choices[0].message.content,
                'model': self.model,
                'tokens': {
                    'prompt': response.usage.prompt_tokens,
                    'completion': response.usage.completion_tokens,
                    'total': response.usage.total_tokens
                },
                'time': elapsed_time,
                'success': True,
                'error': None
            }

        except Exception as e:
            elapsed_time = time.time() - start_time

            return {
                'response': f"Lo siento, hubo un error al generar la respuesta: {str(e)}",
                'model': self.model,
                'tokens': None,
                'time': elapsed_time,
                'success': False,
                'error': str(e)
            }

    def generate_simple(self, query: str, context: str) -> str:
        """
        Versión simplificada que solo retorna el texto de respuesta

        Args:
            query: Pregunta del usuario
            context: Contexto recuperado

        Returns:
            Texto de la respuesta
        """
        result = self.generate(query, context)
        return result['response']

    def get_model_info(self) -> Dict:
        """Retorna información del modelo"""
        return {
            'provider': 'GROQ',
            'model': self.model,
            'temperature': self.temperature,
            'max_tokens': self.max_tokens,
            'location': 'Cloud (GROQ API)'
        }

# ==================== INICIALIZACIÓN ====================

print("\n🔧 Inicializando LLM Generator...\n")

# Crear generador LLM
llm_generator = LLMGenerator(
    api_key=config.GROQ_API_KEY,
    model="llama-3.3-70b-versatile",
    temperature=0.7,
    max_tokens=1024
)

print("\n" + "="*60)
print("✅ LLM GENERATOR LISTO")
print("="*60)

# Mostrar información del modelo
model_info = llm_generator.get_model_info()
print(f"""
Información del modelo:
  - Proveedor: {model_info['provider']}
  - Modelo: {model_info['model']}
  - Ubicación: {model_info['location']}
  - Temperature: {model_info['temperature']}
  - Max tokens: {model_info['max_tokens']}

Justificación de elección:
  ✅ GROQ ofrece inferencia ultra-rápida (>300 tokens/seg)
  ✅ LLama 3.3 70B es un modelo de alta calidad
  ✅ API gratuita con límites generosos
  ✅ Sin filtros restrictivos (a diferencia de Gemini)
  ✅ Ideal para producción y desarrollo

Uso:
  result = llm_generator.generate(query, context)
  print(result['response'])

  # O simplificado:
  response = llm_generator.generate_simple(query, context)
""")

print("="*60 + "\n")


LLM GENERATOR


🔧 Inicializando LLM Generator...

🤖 LLM Generator inicializado
   Proveedor: GROQ
   Modelo: llama-3.3-70b-versatile
   Temperature: 0.7
   Max tokens: 1024

✅ LLM GENERATOR LISTO

Información del modelo:
  - Proveedor: GROQ
  - Modelo: llama-3.3-70b-versatile
  - Ubicación: Cloud (GROQ API)
  - Temperature: 0.7
  - Max tokens: 1024

Justificación de elección:
  ✅ GROQ ofrece inferencia ultra-rápida (>300 tokens/seg)
  ✅ LLama 3.3 70B es un modelo de alta calidad
  ✅ API gratuita con límites generosos
  ✅ Sin filtros restrictivos (a diferencia de Gemini)
  ✅ Ideal para producción y desarrollo

Uso:
  result = llm_generator.generate(query, context)
  print(result['response'])

  # O simplificado:
  response = llm_generator.generate_simple(query, context)




In [ ]:
"""
============================================
CELDA 14: PRUEBAS DEL LLM GENERATOR
============================================
"""

print("\n" + "="*60)
print("PRUEBAS DEL LLM GENERATOR")
print("="*60 + "\n")

# ==================== PRUEBA 1: GENERACIÓN SIMPLE ====================

print("📝 PRUEBA 1: Generación simple con contexto")
print("-" * 60)

query = "¿Cómo uso mi licuadora?"
context = """INFORMACIÓN RECUPERADA (Fuente: vectorial):

1. INSTRUCCIONES DE USO PASO A PASO - Procedimientos de Uso
Para preparar smoothies: 1) Cortar frutas en trozos pequeños, 2) Agregar líquido primero,
3) Agregar frutas congeladas al final, 4) Usar modo pulse para romper hielo...
   [Fuente: manual]

2. INSTRUCCIONES DE USO - Pregunta: ¿Cómo preparar smoothies con la licuadora?
Respuesta: Para mejores resultados, agregue primero los líquidos, luego frutas suaves
y finalmente hielo o frutas congeladas...
   [Fuente: faq]
"""

print(f"Query: '{query}'")
print(f"\nContexto proporcionado: {len(context)} caracteres")
print("\n🤖 Generando respuesta...\n")

result = llm_generator.generate(query, context)

print("="*60)
print("RESPUESTA GENERADA:")
print("="*60)
print(result['response'])
print("="*60)

print(f"\n📊 Estadísticas:")
print(f"  ✅ Éxito: {result['success']}")
print(f"  ⏱️  Tiempo: {result['time']:.2f}s")

if result.get('tokens'):
    print(f"  🔢 Tokens prompt: {result['tokens'].get('prompt', 'N/A')}")
    print(f"  🔢 Tokens completados: {result['tokens'].get('completion', 'N/A')}")
    print(f"  🔢 Tokens totales: {result['tokens'].get('total', 'N/A')}")
else:
    print(f"  🔢 Tokens: N/A (error en la solicitud)")


# ==================== PRUEBA 2: CON INFORMACIÓN INSUFICIENTE ====================

print("\n\n" + "="*60)
print("📝 PRUEBA 2: Respuesta cuando NO hay información suficiente")
print("-" * 60)

query = "¿Cuál es el color del producto P9999?"
context = "No se encontró información relevante para responder esta consulta."

print(f"Query: '{query}'")
print(f"Contexto: '{context}'")
print("\n🤖 Generando respuesta...\n")

result = llm_generator.generate(query, context)

print("="*60)
print("RESPUESTA GENERADA:")
print("="*60)
print(result['response'])
print("="*60)

# ==================== PRUEBA 3: VERSIÓN SIMPLIFICADA ====================

print("\n\n" + "="*60)
print("📝 PRUEBA 3: Uso simplificado (solo respuesta)")
print("-" * 60)

query = "¿Qué productos son compatibles con P0016?"
context = """INFORMACIÓN RECUPERADA (Fuente: grafo):

1. nombre: Procesadora | marca: TechHome | precio: 329.07
2. nombre: Batidora Ultra | marca: ChefMaster | precio: 450.23
3. nombre: Picadora Compacta | marca: TechHome | precio: 189.99
"""

print(f"Query: '{query}'")
print("\n🤖 Generando respuesta simplificada...\n")

response = llm_generator.generate_simple(query, context)

print("="*60)
print("RESPUESTA:")
print("="*60)
print(response)
print("="*60)

# ==================== PRUEBA 4: INTEGRACIÓN CON PIPELINE ====================

print("\n\n" + "="*60)
print("📝 PRUEBA 4: Integración completa (Pipeline + LLM)")
print("-" * 60)

query = "¿Cuáles son las licuadoras de menos de $300?"
print(f"Query: '{query}'")

# Recuperar contexto
print("\n1️⃣ Recuperando contexto con el pipeline...")
retrieval_result = retrieval_pipeline.retrieve(query, top_k=5)

print(f"   ✅ Intención: {retrieval_result['intent']}")
print(f"   ✅ Fuente: {retrieval_result['source']}")
print(f"   ✅ Resultados: {retrieval_result['num_results']}")

# Generar respuesta
print("\n2️⃣ Generando respuesta con LLM...")
llm_result = llm_generator.generate(query, retrieval_result['context'])

print("\n" + "="*60)
print("RESPUESTA FINAL:")
print("="*60)
print(llm_result['response'])
print("="*60)

print(f"\n📊 Estadísticas completas:")
print(f"  Pipeline: {retrieval_result['source']} → {retrieval_result['num_results']} resultados")
print(f"  LLM: {llm_result['tokens']['total']} tokens en {llm_result['time']:.2f}s")

# ==================== FUNCIÓN HELPER ====================

def preguntar(query: str, top_k: int = 5):
    """
    Función helper que integra pipeline + LLM

    Args:
        query: Pregunta del usuario
        top_k: Número de resultados a recuperar
    """
    print(f"\n💬 Pregunta: '{query}'")
    print("-" * 60)

    # Recuperar
    retrieval_result = retrieval_pipeline.retrieve(query, top_k)
    print(f"🔍 Búsqueda: {retrieval_result['source']} ({retrieval_result['num_results']} resultados)")

    # Generar
    print(f"🤖 Generando respuesta...")
    llm_result = llm_generator.generate(query, retrieval_result['context'])

    # Mostrar
    print(f"\n💡 Respuesta:")
    print("="*60)
    print(llm_result['response'])
    print("="*60)
    print(f"\n⏱️  {llm_result['time']:.2f}s | 🔢 {llm_result['tokens']['total']} tokens")

    return llm_result

print("\n\n" + "="*60)
print("✅ PRUEBAS COMPLETADAS")
print("="*60)
print("\nFunción helper disponible:")
print("  preguntar('tu pregunta', top_k=5)")
print("\nEjemplo:")
print("  preguntar('¿Cómo limpiar mi licuadora?')")


PRUEBAS DEL LLM GENERATOR

📝 PRUEBA 1: Generación simple con contexto
------------------------------------------------------------
Query: '¿Cómo uso mi licuadora?'

Contexto proporcionado: 519 caracteres

🤖 Generando respuesta...

RESPUESTA GENERADA:
Para usar tu licuadora, específicamente para preparar smoothies, te recomiendo seguir los siguientes pasos:

1. Corta las frutas en trozos pequeños.
2. Agrega líquido primero.
3. Luego, agrega frutas suaves.
4. Finalmente, agrega frutas congeladas o hielo.
5. Utiliza el modo pulse para romper el hielo.

Estos pasos te ayudarán a obtener mejores resultados al preparar smoothies con tu licuadora.

📊 Estadísticas:
  ✅ Éxito: True
  ⏱️  Tiempo: 0.35s
  🔢 Tokens prompt: 363
  🔢 Tokens completados: 114
  🔢 Tokens totales: 477


📝 PRUEBA 2: Respuesta cuando NO hay información suficiente
------------------------------------------------------------
Query: '¿Cuál es el color del producto P9999?'
Contexto: 'No se encontró información relevante para 

TypeError: 'NoneType' object is not subscriptable

In [ ]:
preguntar("¿Qué marcas de heladeras hay disponibles por sucursal?", top_k=10)

In [ ]:
"""
============================================
CELDA 15: SISTEMA CONVERSACIONAL CON MEMORIA
============================================
Sistema RAG completo con:
- Memoria de conversación
- Integración completa de todos los componentes
- Chat interactivo
"""

from typing import List, Dict, Optional
import time

print("\n" + "="*60)
print("SISTEMA CONVERSACIONAL RAG")
print("="*60 + "\n")

# ==================== MEMORIA CONVERSACIONAL ====================

class ConversationalMemory:
    """Gestiona el historial de conversación"""

    def __init__(self, max_turns: int = 5):
        self.max_turns = max_turns
        self.history = []

        print(f"💭 Memoria conversacional inicializada")
        print(f"   Máximo de turnos: {max_turns}")

    def add_turn(self, user_message: str, assistant_message: str):
        """Agrega un turno de conversación"""
        self.history.append({
            "role": "user",
            "content": user_message
        })
        self.history.append({
            "role": "assistant",
            "content": assistant_message
        })

        # Mantener solo los últimos N turnos
        if len(self.history) > self.max_turns * 2:
            self.history = self.history[-(self.max_turns * 2):]

    def get_history(self) -> List[Dict]:
        """Retorna el historial"""
        return self.history

    def clear(self):
        """Limpia la memoria"""
        self.history = []
        print("🗑️ Memoria limpiada")

    def get_summary(self) -> str:
        """Retorna un resumen del historial"""
        if not self.history:
            return "Memoria vacía"

        turns = len(self.history) // 2
        return f"{turns} turnos en memoria"

# ==================== SISTEMA RAG CONVERSACIONAL ====================

class ConversationalRAG:
    """
    Sistema RAG completo con capacidades conversacionales
    """

    def __init__(
        self,
        retrieval_pipeline,
        llm_generator,
        memory: Optional[ConversationalMemory] = None,
        language: str = 'es'
    ):
        self.retrieval_pipeline = retrieval_pipeline
        self.llm_generator = llm_generator
        self.memory = memory or ConversationalMemory(max_turns=5)
        self.language = language

        print("🤖 Sistema RAG Conversacional inicializado")
        print(f"   Idioma: {language}")
        print(f"   Memoria: {self.memory.max_turns} turnos")

    def chat(self, user_message: str, top_k: int = 5, verbose: bool = True) -> Dict:
        """
        Procesa un mensaje del usuario y genera respuesta

        Args:
            user_message: Mensaje del usuario
            top_k: Número de resultados a recuperar
            verbose: Mostrar información de debug

        Returns:
            Dict con: response, intent, source, time, etc.
        """

        start_time = time.time()

        if verbose:
            print(f"\n💬 Usuario: {user_message}")
            print("-" * 60)

        # 1. Recuperar información relevante
        if verbose:
            print("🔍 Recuperando información...")

        retrieval_result = self.retrieval_pipeline.retrieve(user_message, top_k)

        if verbose:
            print(f"   Intención: {retrieval_result['intent']}")
            print(f"   Fuente: {retrieval_result['source']}")
            print(f"   Resultados: {retrieval_result['num_results']}")

        # 2. Generar respuesta con historial conversacional
        if verbose:
            print("🤖 Generando respuesta...")

        llm_result = self.llm_generator.generate(
            query=user_message,
            context=retrieval_result['context'],
            conversational_history=self.memory.get_history()
        )

        # 3. Actualizar memoria
        self.memory.add_turn(user_message, llm_result['response'])

        # 4. Calcular tiempo total
        total_time = time.time() - start_time

        if verbose:
            print(f"\n⏱️  Tiempo total: {total_time:.2f}s")
            print(f"🔢 Tokens: {llm_result['tokens']['total']}")

        return {
            'response': llm_result['response'],
            'intent': retrieval_result['intent'],
            'source': retrieval_result['source'],
            'num_results': retrieval_result['num_results'],
            'tokens': llm_result['tokens'],
            'time': total_time,
            'success': llm_result['success']
        }

    def chat_simple(self, user_message: str, top_k: int = 5) -> str:
        """Versión simplificada que solo retorna la respuesta"""
        result = self.chat(user_message, top_k, verbose=False)
        return result['response']

    def reset_memory(self):
        """Reinicia la memoria conversacional"""
        self.memory.clear()

    def get_memory_summary(self) -> str:
        """Obtiene resumen de la memoria"""
        return self.memory.get_summary()

    def print_response(self, result: Dict):
        """Imprime una respuesta de forma bonita"""
        print("\n" + "="*60)
        print("🤖 ASISTENTE:")
        print("="*60)
        print(result['response'])
        print("="*60)
        print(f"\n📊 Fuente: {result['source']} | ⏱️ {result['time']:.2f}s | 🔢 {result['tokens']['total']} tokens")

# ==================== INICIALIZACIÓN ====================

print("🔧 Inicializando sistema conversacional...\n")

# Crear sistema RAG conversacional
rag_system = ConversationalRAG(
    retrieval_pipeline=retrieval_pipeline,
    llm_generator=llm_generator,
    memory=ConversationalMemory(max_turns=5),
    language='es'
)

print("\n" + "="*60)
print("✅ SISTEMA CONVERSACIONAL RAG LISTO")
print("="*60)
print("""
Componentes integrados:
  ✅ Pipeline de recuperación (3 fuentes)
  ✅ Clasificador de intención
  ✅ LLM Generator (GROQ)
  ✅ Memoria conversacional (5 turnos)

Uso básico:
  # Chat interactivo
  result = rag_system.chat("¿Cómo usar mi licuadora?")

  # Solo respuesta
  response = rag_system.chat_simple("¿Qué productos hay de menos de $200?")

  # Resetear memoria
  rag_system.reset_memory()

Características:
  - Mantiene contexto de conversación
  - Soporta preguntas de seguimiento
  - Responde en español
  - Si no hay información, sugiere reformular
""")
print("="*60 + "\n")

In [ ]:
"""
============================================
CELDA 16: PRUEBAS DEL SISTEMA CONVERSACIONAL
============================================
Pruebas de conversaciones multi-turno con memoria
"""

print("\n" + "="*60)
print("PRUEBAS DEL SISTEMA CONVERSACIONAL")
print("="*60 + "\n")

# ==================== PRUEBA 1: CONVERSACIÓN SIMPLE ====================

print("📝 PRUEBA 1: Conversación simple (sin memoria)")
print("="*60)

result = rag_system.chat("¿Cómo usar mi licuadora para hacer smoothies?")
rag_system.print_response(result)

# ==================== PRUEBA 2: CONVERSACIÓN CON MEMORIA ====================

print("\n\n" + "="*60)
print("📝 PRUEBA 2: Conversación con memoria (seguimiento)")
print("="*60)

# Resetear memoria
rag_system.reset_memory()

# Turno 1
print("\n--- Turno 1 ---")
result1 = rag_system.chat("¿Cuáles son las licuadoras de menos de $300?")
rag_system.print_response(result1)

# Turno 2 (pregunta de seguimiento)
print("\n--- Turno 2 (seguimiento) ---")
result2 = rag_system.chat("¿Cuál de esas tiene mejor garantía?")
rag_system.print_response(result2)

print(f"\n💭 Memoria: {rag_system.get_memory_summary()}")

# ==================== PRUEBA 3: CONVERSACIÓN MULTI-FUENTE ====================

print("\n\n" + "="*60)
print("📝 PRUEBA 3: Conversación alternando fuentes")
print("="*60)

# Resetear memoria
rag_system.reset_memory()

# Turno 1 - Vectorial
print("\n--- Turno 1 (Vectorial) ---")
result1 = rag_system.chat("¿Cómo limpiar una licuadora?")
rag_system.print_response(result1)

# Turno 2 - Tabular
print("\n--- Turno 2 (Tabular) ---")
result2 = rag_system.chat("¿Cuánto cuesta una licuadora TechHome?")
rag_system.print_response(result2)

# Turno 3 - Grafo
print("\n--- Turno 3 (Grafo) ---")
result3 = rag_system.chat("¿Qué productos son compatibles con P0016?")
rag_system.print_response(result3)

print(f"\n💭 Memoria: {rag_system.get_memory_summary()}")

# ==================== PRUEBA 4: CUANDO NO HAY INFORMACIÓN ====================

print("\n\n" + "="*60)
print("📝 PRUEBA 4: Sin información disponible")
print("="*60)

result = rag_system.chat("¿Cuál es el color favorito del CEO de TechHome?")
rag_system.print_response(result)

# ==================== PRUEBA 5: BATCH DE PREGUNTAS ====================

print("\n\n" + "="*60)
print("📝 PRUEBA 5: Batch de preguntas (para el informe)")
print("="*60)

# Resetear memoria para cada conversación
test_queries = [
    "¿Cómo usar mi licuadora para hacer smoothies?",
    "¿Cuáles son las licuadoras de menos de $200?",
    "¿Qué productos son compatibles con P0016?",
    "¿Qué opinan los usuarios de las cafeteras?",
    "Productos con garantía mayor a 24 meses"
]

results_summary = []

for i, query in enumerate(test_queries, 1):
    print(f"\n--- Pregunta {i}/5 ---")
    rag_system.reset_memory()

    result = rag_system.chat(query, verbose=True)

    results_summary.append({
        'query': query,
        'intent': result['intent'],
        'source': result['source'],
        'num_results': result['num_results'],
        'time': result['time'],
        'tokens': result['tokens']['total'],
        'success': result['success']
    })

    print(f"\n💡 Respuesta: {result['response'][:150]}...")

# Resumen
print("\n\n" + "="*60)
print("📊 RESUMEN DE RESULTADOS (para el informe)")
print("="*60)

import pandas as pd
df_results = pd.DataFrame(results_summary)

print("\n" + df_results.to_string(index=False))

print(f"\n📈 Estadísticas:")
print(f"  Total consultas: {len(results_summary)}")
print(f"  Exitosas: {sum(1 for r in results_summary if r['success'])}")
print(f"  Tiempo promedio: {df_results['time'].mean():.2f}s")
print(f"  Tokens promedio: {df_results['tokens'].mean():.0f}")

print(f"\n📊 Distribución por fuente:")
print(df_results['source'].value_counts().to_string())

# ==================== FUNCIÓN CHAT INTERACTIVA ====================

def chat_interactivo():
    """
    Inicia una sesión de chat interactiva
    Escribe 'salir' para terminar
    """
    print("\n" + "="*60)
    print("💬 CHAT INTERACTIVO")
    print("="*60)
    print("\nComandos especiales:")
    print("  'salir' - Terminar chat")
    print("  'reset' - Limpiar memoria")
    print("  'memoria' - Ver estado de memoria")
    print("\nEscribe tu pregunta:\n")

    rag_system.reset_memory()

    while True:
        user_input = input("👤 Tú: ").strip()

        if not user_input:
            continue

        if user_input.lower() in ['salir', 'exit', 'quit']:
            print("\n👋 ¡Hasta luego!")
            break

        if user_input.lower() == 'reset':
            rag_system.reset_memory()
            continue

        if user_input.lower() == 'memoria':
            print(f"💭 {rag_system.get_memory_summary()}")
            continue

        result = rag_system.chat(user_input, verbose=False)
        print(f"\n🤖 Asistente: {result['response']}\n")

print("\n\n" + "="*60)
print("✅ PRUEBAS COMPLETADAS")
print("="*60)
print("\nFunción disponible:")
print("  chat_interactivo()  # Inicia chat interactivo")
print("\nEjemplo:")
print("  chat_interactivo()")

In [ ]:
"""
============================================
CELDA 17: HERRAMIENTAS PARA AGENTE REACT
============================================
Encapsula las búsquedas en herramientas de Langchain
- doc_search: Búsqueda en documentos (vectorial)
- table_search: Búsqueda en tablas (Pandas)
- graph_search: Búsqueda en grafos (Neo4j)
- analytics_tool: Análisis y gráficos con SQL/matplotlib
"""

print("\n" + "="*60)
print("HERRAMIENTAS PARA AGENTE REACT")
print("="*60 + "\n")

# Instalar Langchain si no está instalado
print("📦 Verificando dependencias de Langchain...")
try:
    import langchain_groq
    print("   ✅ Langchain-groq ya instalado")
except:
    print("   📦 Instalando Langchain y dependencias...")
    !pip install -q langchain langchain-groq langchain-community langchain-core matplotlib
    print("   ✅ Langchain instalado")

from langchain_core.tools import tool
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
import matplotlib.pyplot as plt
import io
import base64
from typing import Optional

# ==================== HERRAMIENTA 1: DOC_SEARCH ====================

@tool
def doc_search(query: str) -> str:
    """Busca en documentos de texto (manuales, FAQs, reseñas) usando búsqueda híbrida.

    Usa esta herramienta para preguntas sobre:
    - Cómo usar productos
    - Instrucciones de uso
    - Problemas y soluciones
    - Opiniones de usuarios
    - Mantenimiento

    Ejemplo: ¿Cómo usar mi licuadora para hacer smoothies?

    Args:
        query: Consulta del usuario

    Returns:
        Texto con los documentos más relevantes
    """
    try:
        results = hybrid_search.search(query, top_k=5)

        if not results:
            return "No se encontraron documentos relevantes para esta consulta."

        output = f"Encontrados {len(results)} documentos relevantes:\n\n"

        for i, result in enumerate(results, 1):
            source = result['metadata'].get('source', 'N/A')
            text = result['text'][:200]
            output += f"{i}. [{source.upper()}] {text}...\n\n"

        return output.strip()

    except Exception as e:
        return f"Error en búsqueda de documentos: {str(e)}"

# ==================== HERRAMIENTA 2: TABLE_SEARCH ====================

@tool
def table_search(query: str) -> str:
    """Busca en tablas de datos (productos, ventas, inventario) con filtros dinámicos.

    Usa esta herramienta para preguntas sobre:
    - Precios de productos
    - Filtros por características (voltaje, potencia, color, etc.)
    - Stock disponible
    - Especificaciones técnicas
    - Información de ventas

    Ejemplo: ¿Cuáles son las licuadoras de menos de $200?

    Args:
        query: Consulta del usuario

    Returns:
        Resultados de la tabla en formato texto
    """
    try:
        results = table_searcher.search(query, 'productos', max_results=10)

        if len(results) == 0:
            results = table_searcher.search(query, 'ventas', max_results=10)

        if len(results) == 0 and 'inventario' in dataframes:
            results = table_searcher.search(query, 'inventario', max_results=10)

        if len(results) == 0:
            return "No se encontraron resultados en las tablas para esta consulta."

        import pandas as pd
        if isinstance(results, pd.DataFrame):
            cols_to_show = list(results.columns)[:6]
            output = f"Encontrados {len(results)} registros:\n\n"
            output += results[cols_to_show].head(10).to_string(index=False)

            if len(results) > 10:
                output += f"\n\n... y {len(results) - 10} registros más"

            return output
        else:
            return str(results)

    except Exception as e:
        return f"Error en búsqueda de tablas: {str(e)}"

# ==================== HERRAMIENTA 3: GRAPH_SEARCH ====================

@tool
def graph_search(query: str) -> str:
    """Busca relaciones en el grafo de conocimiento (Neo4j) con Cypher dinámico.

    Usa esta herramienta para preguntas sobre:
    - Productos compatibles
    - Productos relacionados
    - Accesorios
    - Stock por sucursal
    - Productos de misma marca/categoría

    Ejemplo: ¿Qué productos son compatibles con P0016?

    Args:
        query: Consulta del usuario

    Returns:
        Relaciones encontradas en formato texto
    """
    try:
        results = graph_searcher.search(query)

        if not results:
            return "No se encontraron relaciones en el grafo para esta consulta."

        output = f"Encontradas {len(results)} relaciones:\n\n"

        for i, result in enumerate(results[:10], 1):
            items = " | ".join([f"{k}: {v}" for k, v in result.items()])
            output += f"{i}. {items}\n"

        if len(results) > 10:
            output += f"\n... y {len(results) - 10} relaciones más"

        return output.strip()

    except Exception as e:
        return f"Error en búsqueda de grafos: {str(e)}"

# ==================== HERRAMIENTA 4: ANALYTICS_TOOL ====================

@tool
def analytics_tool(query: str) -> str:
    """Realiza análisis de datos y genera gráficos con matplotlib.

    Usa esta herramienta para:
    - Distribuciones (métodos de pago, ventas por sucursal)
    - Agregaciones (total ventas, promedios)
    - Top productos
    - Estadísticas generales

    Ejemplo: Dame un gráfico sobre la distribución de métodos de pago

    Args:
        query: Descripción del análisis deseado

    Returns:
        Resultado del análisis en texto
    """
    try:
        query_lower = query.lower()

        if 'gráfico' in query_lower or 'grafico' in query_lower or 'distribución' in query_lower:

            if 'método de pago' in query_lower or 'metodo de pago' in query_lower:
                if 'ventas' in dataframes:
                    df = dataframes['ventas']
                    counts = df['metodo_pago'].value_counts()

                    plt.figure(figsize=(10, 6))
                    counts.plot(kind='pie', autopct='%1.1f%%')
                    plt.title('Distribución de Métodos de Pago')
                    plt.ylabel('')

                    output_path = config.OUTPUT_DIR / 'metodos_pago.png'
                    plt.savefig(output_path, bbox_inches='tight', dpi=150)
                    plt.close()

                    output = f"Análisis de métodos de pago:\n\n"
                    for metodo, cantidad in counts.items():
                        pct = (cantidad / counts.sum()) * 100
                        output += f"  - {metodo}: {cantidad} ({pct:.1f}%)\n"

                    output += f"\n✅ Gráfico guardado en: {output_path}"

                    return output

            elif 'sucursal' in query_lower or 'ventas por sucursal' in query_lower:
                if 'ventas' in dataframes:
                    df = dataframes['ventas']
                    ventas_sucursal = df.groupby('sucursal')['total'].sum().sort_values(ascending=False)

                    plt.figure(figsize=(10, 6))
                    ventas_sucursal.plot(kind='bar')
                    plt.title('Ventas Totales por Sucursal')
                    plt.xlabel('Sucursal')
                    plt.ylabel('Total Ventas (USD)')
                    plt.xticks(rotation=45, ha='right')
                    plt.tight_layout()

                    output_path = config.OUTPUT_DIR / 'ventas_sucursal.png'
                    plt.savefig(output_path, bbox_inches='tight', dpi=150)
                    plt.close()

                    output = f"Análisis de ventas por sucursal:\n\n"
                    for sucursal, total in ventas_sucursal.items():
                        output += f"  - {sucursal}: ${total:,.2f}\n"

                    output += f"\n✅ Gráfico guardado en: {output_path}"

                    return output

        elif 'total' in query_lower or 'suma' in query_lower or 'cuánto' in query_lower:
            if 'ventas' in dataframes:
                df = dataframes['ventas']
                total = df['total'].sum()
                cantidad = len(df)
                promedio = df['total'].mean()

                return f"""Análisis de ventas:
  - Total de ventas: ${total:,.2f}
  - Cantidad de transacciones: {cantidad:,}
  - Promedio por transacción: ${promedio:,.2f}"""

        elif 'top' in query_lower or 'mejores' in query_lower:
            if 'ventas' in dataframes:
                df = dataframes['ventas']
                top_productos = df.groupby('id_producto').agg({
                    'cantidad': 'sum',
                    'total': 'sum'
                }).sort_values('cantidad', ascending=False).head(10)

                output = "Top 10 productos más vendidos:\n\n"
                for i, (prod_id, row) in enumerate(top_productos.iterrows(), 1):
                    output += f"{i}. {prod_id}: {row['cantidad']} unidades (${row['total']:,.2f})\n"

                return output

        return "No se pudo realizar el análisis solicitado. Intenta especificar mejor qué tipo de análisis necesitas."

    except Exception as e:
        return f"Error en análisis de datos: {str(e)}"

# ==================== CREAR LISTA DE HERRAMIENTAS ====================

print("🔧 Creando lista de herramientas de Langchain...\n")

# Las herramientas ya están definidas con el decorador @tool arriba
tools = [doc_search, table_search, graph_search, analytics_tool]

print("✅ Herramientas creadas:")
for tool in tools:
    print(f"   - {tool.name}")

print("\n" + "="*60)
print("✅ HERRAMIENTAS PARA AGENTE REACT LISTAS")
print("="*60)
print("""
4 herramientas disponibles:
  1. doc_search - Búsqueda en documentos (vectorial)
  2. table_search - Búsqueda en tablas (Pandas)
  3. graph_search - Búsqueda en grafos (Neo4j)
  4. analytics_tool - Análisis y gráficos

Las herramientas están listas para ser usadas por el agente ReAct.
""")
print("="*60 + "\n")

In [ ]:
"""
============================================
CELDA 18: AGENTE REACT CON LANGCHAIN
============================================
Agente inteligente que razona (Thought), actúa (Action) y observa (Observation)
para responder consultas complejas usando múltiples herramientas
"""

import time
import re

print("\n" + "="*60)
print("AGENTE REACT CON LANGCHAIN")
print("="*60 + "\n")

# ==================== CONFIGURAR LLM ====================

print("🤖 Configurando LLM para el agente...")

# Usar GROQ con llama-3.3-70b
llm = ChatGroq(
    api_key=config.GROQ_API_KEY,
    model_name="llama-3.3-70b-versatile",
    temperature=0.1,
    max_tokens=2048
)

print("   ✅ LLM configurado: llama-3.3-70b-versatile")

# ==================== AGENTE REACT MANUAL ====================

class SimpleReActAgent:
    """
    Agente ReAct implementado manualmente
    Sigue el paradigma: Thought → Action → Observation → Final Answer
    """

    def __init__(self, llm, tools, max_iterations=5):
        self.llm = llm
        self.tools = {tool.name: tool for tool in tools}
        self.max_iterations = max_iterations

        # Crear descripción de herramientas
        self.tools_desc = "\n".join([
            f"- {name}: {tool.description}"
            for name, tool in self.tools.items()
        ])

    def _create_prompt(self, query: str, history: str = "") -> str:
        """Crea el prompt ReAct"""

        prompt = f"""Eres un asistente virtual experto en electrodomésticos.

Tienes acceso a estas herramientas:

{self.tools_desc}

FORMATO EXACTO que DEBES seguir:

Thought: [tu razonamiento sobre qué hacer]
Action: [nombre_exacto_de_herramienta]
Action Input: [entrada para la herramienta]

O si ya tienes la respuesta:

Thought: Ya tengo suficiente información
Final Answer: [tu respuesta final en español]

REGLAS:
1. Usa el formato EXACTO de arriba
2. Nombres de herramientas: doc_search, table_search, graph_search, analytics_tool
3. Responde SIEMPRE en español
4. Piensa paso a paso

Pregunta: {query}

{history}

Comienza con "Thought:"""

        return prompt

    def run(self, query: str, verbose: bool = True) -> dict:
        """Ejecuta el agente"""

        start_time = time.time()
        history = ""
        intermediate_steps = []

        if verbose:
            print(f"\n💬 Usuario: {query}")
            print("="*60)
            print("🤖 Agente ReAct procesando...\n")

        for iteration in range(self.max_iterations):
            if verbose:
                print(f"--- Iteración {iteration + 1} ---")

            # Generar respuesta del LLM
            prompt = self._create_prompt(query, history)

            try:
                response = self.llm.invoke(prompt)
                response_text = response.content

                if verbose:
                    print(response_text)
                    print()

                # Buscar Final Answer
                if "Final Answer:" in response_text:
                    match = re.search(r'Final Answer:\s*(.+)', response_text, re.DOTALL)
                    if match:
                        final_answer = match.group(1).strip()
                        elapsed_time = time.time() - start_time

                        return {
                            'output': final_answer,
                            'intermediate_steps': intermediate_steps,
                            'time': elapsed_time,
                            'success': True,
                            'error': None
                        }

                # Extraer Action y Action Input
                action_match = re.search(r'Action:\s*(\w+)', response_text)
                input_match = re.search(r'Action Input:\s*(.+?)(?:\n|$)', response_text, re.DOTALL)

                if action_match and input_match:
                    action = action_match.group(1).strip()
                    action_input = input_match.group(1).strip()

                    # Ejecutar herramienta
                    if action in self.tools:
                        tool = self.tools[action]
                        observation = tool.func(action_input)

                        intermediate_steps.append({
                            'action': action,
                            'input': action_input,
                            'observation': observation
                        })

                        # Agregar a historial
                        history += f"\n{response_text}\nObservation: {observation}\n"

                        if verbose:
                            print(f"Observation: {observation}\n")
                    else:
                        if verbose:
                            print(f"⚠️ Herramienta '{action}' no encontrada\n")
                        history += f"\n{response_text}\nObservation: Error - Herramienta no encontrada\n"
                else:
                    # No se pudo parsear, intentar de nuevo
                    if verbose:
                        print("⚠️ No se pudo parsear Action/Action Input\n")
                    break

            except Exception as e:
                elapsed_time = time.time() - start_time
                return {
                    'output': f"Error en iteración {iteration + 1}: {str(e)}",
                    'intermediate_steps': intermediate_steps,
                    'time': elapsed_time,
                    'success': False,
                    'error': str(e)
                }

        # Si llegamos aquí, no se encontró respuesta final
        elapsed_time = time.time() - start_time
        return {
            'output': "No se pudo completar la consulta en el número máximo de iteraciones.",
            'intermediate_steps': intermediate_steps,
            'time': elapsed_time,
            'success': False,
            'error': 'Max iterations reached'
        }

print("\n🔧 Creando agente ReAct...")

# Crear agente
agent_executor = SimpleReActAgent(llm, tools, max_iterations=5)

print("   ✅ Agente ReAct creado con éxito")

# ==================== WRAPPER PARA USO FÁCIL ====================

class ReactAgent:
    """Wrapper del agente ReAct para uso simplificado"""

    def __init__(self, executor):
        self.executor = executor

    def run(self, query: str, verbose: bool = True) -> dict:
        """Ejecuta el agente"""
        return self.executor.run(query, verbose)

    def run_simple(self, query: str) -> str:
        """Versión simplificada que solo retorna la respuesta"""
        result = self.run(query, verbose=False)
        return result['output']

    def print_response(self, result: dict):
        """Imprime una respuesta de forma bonita"""
        print("\n" + "="*60)
        print("🎯 RESPUESTA FINAL:")
        print("="*60)
        print(result['output'])
        print("="*60)

        steps = len(result.get('intermediate_steps', []))
        print(f"\n📊 Herramientas usadas: {steps} | ⏱️ Tiempo: {result['time']:.2f}s")

# Crear instancia del agente
react_agent = ReactAgent(agent_executor)

print("\n" + "="*60)
print("✅ AGENTE REACT LISTO")
print("="*60)
print("""
Características:
  ✅ Paradigma ReAct (Thought → Action → Observation)
  ✅ 4 herramientas disponibles
  ✅ Razonamiento paso a paso
  ✅ Puede usar múltiples herramientas en una consulta
  ✅ Implementación manual sin dependencias complejas

Uso:
  # Con verbose (muestra razonamiento)
  result = react_agent.run("tu consulta")

  # Sin verbose (solo respuesta)
  response = react_agent.run_simple("tu consulta")

  # Imprimir bonito
  react_agent.print_response(result)

Ejemplos de consultas:
  - "¿Cómo usar mi licuadora para hacer smoothies?"
  - "¿Cuáles son las licuadoras de menos de $200?"
  - "Dame un gráfico de ventas por sucursal"
  - "¿Qué productos son compatibles con P0016?"
""")
print("="*60 + "\n")

In [ ]:
# Consulta vectorial (doc_search)
react_agent.run("¿Cómo limpiar mi licuadora?")

# Consulta tabular (table_search)
react_agent.run("Productos de menos de $300")

# Consulta grafo (graph_search)
react_agent.run("¿Dónde hay stock del producto P0001?")

# Consulta analytics (analytics_tool)
react_agent.run("Dame un gráfico de ventas por sucursal")

# Consulta compleja (múltiples herramientas)
react_agent.run("¿Qué productos compatibles con P0016 cuestan menos de $300?")